In [1]:
# need to set some versions
import sys
!pip install imblearn

# For Google Colab:
#Mounting drive to access datafile
# from google.colab import drive
# drive.mount('/content/drive')

# for Google Workbook:
!{sys.executable} -m pip install --user pandas==1.1.5
!{sys.executable} -m pip install --user tensorflow==1.13.1 # 1.15.0 # paper said us 1.10.0, but it doesn't exist???
!{sys.executable} -m pip install --user numpy==1.19.5  # this one breaks on the tensor to numpy issue: 1.16.0 # tested with 1.16.0 # paper said us 1.15.3, but too many dependency conflicts
!{sys.executable} -m pip install --user hmmlearn

# for other environments:
# !{sys.executable} -m pip install pandas==1.1.5
# !{sys.executable} -m pip install tensorflow==1.13.1 # 1.15.0 # paper said us 1.10.0, but it doesn't exist???
# !{sys.executable} -m pip install numpy==1.19.5  # this one breaks on the tensor to numpy issue: 1.16.0 # tested with 1.16.0 # paper said us 1.15.3, but too many dependency conflicts
# !{sys.executable} -m pip install hmmlearn

# may have to restart kernel after running these to get access to them

  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)
  Using cached imbalanced_learn-0.9.0-py3-none-any.whl (199 kB)


In [ ]:
#Importing/installing modules
from __future__ import absolute_import, division, print_function

import collections
import hashlib
import numbers
import itertools
import functools
import inspect
import pickle

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
get_ipython().magic('matplotlib inline')

import warnings
warnings.filterwarnings("ignore")

import os
from pathlib import Path
import shutil

import tensorflow as tf
from tensorflow.contrib.rnn import PhasedLSTMCell, MultiRNNCell, BasicRNNCell
from tensorflow.python.ops import rnn_cell, rnn
from tensorflow.python.eager import context
from tensorflow.python.framework import constant_op, dtypes, ops, tensor_shape, tensor_util   
from tensorflow.python.layers import base as base_layer
from tensorflow.python.ops import * 
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.training import checkpointable
from tensorflow.python.util import nest
from tensorflow.python.util.tf_export import tf_export

from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import mean_absolute_error
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal
from hmmlearn import hmm

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [ ]:
pd.__version__

In [ ]:
np.__version__

In [ ]:
tf.__version__

In [ ]:
# Python version - should be 3.6 or could have issues
print(sys.executable)
print(sys.version)
print(sys.version_info)

In [ ]:
# pull up data for EDA
heal_df = pd.read_csv("CSV_DATA/DAP_2020_HLES_health_condition_v1.1.csv")
ownr_df = pd.read_csv("CSV_DATA/DAP_2020_HLES_dog_owner_v1.1.csv")
cncr_df = pd.read_csv("CSV_DATA/DAP_2020_HLES_cancer_condition_v1.1.csv")

# if in Google Colab:
# heal_df = pd.read_csv("/content/drive/Shared drives/CAPSTONE/CSV_DATA/DAP_2020_HLES_health_condition_v1.1.csv")

In [ ]:
# MINOR EDA
print(len(heal_df))
print(list(heal_df.columns))
heal_df.head(10)

In [ ]:
# need to know how many records per dog
# going to handle each entry as a separate visit for now, 
# but it looks like it is an entry per disease, so should massage later
# When come back to this: assume that if same month and same year, 
# but different diseases, it was during the same visit
heal_df.groupby('dog_id').count()

In [ ]:
seq_length = 4
# reduce to just dogs with 4+ health records
# then look at the most common conditions & most common surgical/hospitalization scenarios
sm_hl_df = heal_df[['dog_id', 'hs_condition_type','hs_required_surgery_or_hospitalization']]
# sm_hl_df.groupby('hs_condition_type').count()
# reduce to only dogs with seq_lenth+ of records
segmin_df = sm_hl_df[sm_hl_df.groupby('dog_id')['dog_id'].transform('size') > seq_length]
print(len(segmin_df))
# most common conditions
segmin_df.groupby('hs_condition_type').count()
# most common are (in order):
# 4, 3, 18, 16, 11, 1, etc.
# skin - 4 - 4990
# mouth/dental/oral - 3 - 4837
# trauma - 18 - 3231
# infection/parasites - 16 - 3197
# bone/orthoepedic - 11 - 2837
# eye - 1 - 2170

In [ ]:
kicked = heal_df.loc[(heal_df['hs_condition']==1811),['dog_id', 'hs_condition_type','hs_required_surgery_or_hospitalization']]
kicked

In [ ]:
skin = heal_df.loc[(heal_df['hs_condition_type']==4),['dog_id', 'hs_condition','hs_required_surgery_or_hospitalization']]
skin

In [ ]:
skin.groupby('hs_condition').count()

In [ ]:
# just looking at skin issues - want it broken down by hospitalized, surgery, both, none
skin1_df = heal_df[['dog_id', 'hs_condition_type','hs_required_surgery_or_hospitalization']]
# sm_hl_df.groupby('hs_condition_type').count()
# reduce to only dogs with seq_lenth+ of records
# segmin_df = skin1_df[skin1_df.groupby('dog_id')['dog_id'].transform('size') > seq_length]
print(len(skin1_df))
skin2_df = skin1_df.loc[(skin1_df['hs_condition_type']==3),['dog_id', 'hs_condition_type','hs_required_surgery_or_hospitalization']]
# most common conditions
skin2_df.groupby('hs_required_surgery_or_hospitalization').count()

In [ ]:
# most common surgical/hospitalization scenarios
segmin_df.groupby('hs_required_surgery_or_hospitalization').count()

In [ ]:
print(len(cncr_df))
print(list(cncr_df.columns))
cncr_df # .head(10)

In [ ]:
print(len(ownr_df))
print(list(ownr_df.columns))
ownr_df.head(10)

# Time-series Generative Adversarial Networks - MODEL CODE

In [ ]:
### TimeGANs key functionality
## Min Max Normalizer

def MinMaxScaler2(dataX):
    
    min_val = np.min(np.min(dataX, axis = 0), axis = 0)
    dataX = dataX - min_val
    
    max_val = np.max(np.max(dataX, axis = 0), axis = 0)
    dataX = dataX / (max_val + 1e-7)
    
    return dataX, min_val, max_val

## Start TimeGAN function (Input: Original data, Output: Synthetic Data)

def timegan (dataX, parameters):
  
    # Initialization on the Graph
    tf.compat.v1.reset_default_graph()

    # Basic Parameters
    No = len(dataX)
    data_dim = len(dataX[0][0,:])
    
    # Maximum seq length and each seq length
    dataT = list()
    Max_Seq_Len = 0
    for i in range(No):
        Max_Seq_Len = max(Max_Seq_Len, len(dataX[i][:,0]))
        dataT.append(len(dataX[i][:,0]))
        
    # Normalization
    if ((np.max(dataX) > 1) | (np.min(dataX) < 0)):
        dataX, min_val, max_val = MinMaxScaler2(dataX)
        Normalization_Flag = 1
    else:
        Normalization_Flag = 0
     
    # Network Parameters
    hidden_dim   = parameters['hidden_dim'] 
    num_layers   = parameters['num_layers']
    iterations   = parameters['iterations']
    batch_size   = parameters['batch_size']
    module_name  = parameters['module_name']    # 'lstm' or 'lstmLN'
    z_dim        = parameters['z_dim']
    gamma        = 1
    
    ## input place holders
    tf.compat.v1.disable_eager_execution()
    X = tf.compat.v1.placeholder(tf.float32, [None, Max_Seq_Len, data_dim], name = "myinput_x")
    Z = tf.compat.v1.placeholder(tf.float32, [None, Max_Seq_Len, z_dim], name = "myinput_z")
    T = tf.compat.v1.placeholder(tf.int32, [None], name = "myinput_t")
    
    ## Basic RNN Cell
          
    def rnn_cell(module_name):
      # GRU
        if (module_name == 'gru'):
            rnn_cell = tf.compat.v1.nn.rnn_cell.GRUCell(num_units=hidden_dim, activation=tf.nn.tanh)
      # LSTM
        elif (module_name == 'lstm'):
            rnn_cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(num_units=hidden_dim, activation=tf.nn.tanh)
      # LSTM Layer Normalization
        elif (module_name == 'lstmLN'):
            rnn_cell = tf.contrib.rnn.LayerNormBasicLSTMCell(num_units=hidden_dim, activation=tf.nn.tanh)
        return rnn_cell
      
        
    ## build a RNN embedding network      
    
    def embedder (X, T):      
      
        with tf.compat.v1.variable_scope("embedder", reuse = tf.compat.v1.AUTO_REUSE):
            
            e_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([rnn_cell(module_name) for _ in range(num_layers)])
                
            e_outputs, e_last_states = tf.compat.v1.nn.dynamic_rnn(e_cell, X, dtype=tf.float32, sequence_length = T)
            
            H = tf.contrib.layers.fully_connected(e_outputs, hidden_dim, activation_fn=tf.nn.sigmoid)

        return H
      
    ##### Recovery
    
    def recovery (H, T):      
      
        with tf.compat.v1.variable_scope("recovery", reuse = tf.compat.v1.AUTO_REUSE):       
              
            r_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([rnn_cell(module_name) for _ in range(num_layers)])
                
            r_outputs, r_last_states = tf.compat.v1.nn.dynamic_rnn(r_cell, H, dtype=tf.float32, sequence_length = T)
            
            X_tilde = tf.contrib.layers.fully_connected(r_outputs, data_dim, activation_fn=tf.nn.sigmoid) 

        return X_tilde
    
    
    
    ## build a RNN generator network
    
    def generator (Z, T):      
      
        with tf.compat.v1.variable_scope("generator", reuse = tf.compat.v1.AUTO_REUSE):
            
            e_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([rnn_cell(module_name) for _ in range(num_layers)])
                
            e_outputs, e_last_states = tf.compat.v1.nn.dynamic_rnn(e_cell, Z, dtype=tf.float32, sequence_length = T)
            
            E = tf.contrib.layers.fully_connected(e_outputs, hidden_dim, activation_fn=tf.nn.sigmoid)     

        return E
      
    def supervisor (H, T):      
      
        with tf.compat.v1.variable_scope("supervisor", reuse = tf.compat.v1.AUTO_REUSE):
            
            e_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([rnn_cell(module_name) for _ in range(num_layers-1)])
                
            e_outputs, e_last_states = tf.compat.v1.nn.dynamic_rnn(e_cell, H, dtype=tf.float32, sequence_length = T)
            
            S = tf.contrib.layers.fully_connected(e_outputs, hidden_dim, activation_fn=tf.nn.sigmoid)     

        return S
      
      
      
    ## builde a RNN discriminator network 
    
    def discriminator (H, T):
      
        with tf.compat.v1.variable_scope("discriminator", reuse = tf.compat.v1.AUTO_REUSE):
            
            d_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([rnn_cell(module_name) for _ in range(num_layers)])
                
            d_outputs, d_last_states = tf.compat.v1.nn.dynamic_rnn(d_cell, H, dtype=tf.float32, sequence_length = T)
            
            Y_hat = tf.contrib.layers.fully_connected(d_outputs, 1, activation_fn=None) 
    
        return Y_hat   
    
    
    ## Random vector generation
    def random_generator (batch_size, z_dim, T_mb, Max_Seq_Len):
      
        Z_mb = list()
        
        for i in range(batch_size):
            
            Temp = np.zeros([Max_Seq_Len, z_dim])
            
            Temp_Z = np.random.uniform(0., 1, [T_mb[i], z_dim])
        
            Temp[:T_mb[i],:] = Temp_Z
            
            Z_mb.append(Temp_Z)
      
        return Z_mb
    
    ## Functions
    
    # Embedder Networks
    H = embedder(X, T)
    X_tilde = recovery(H, T)
    
    # Generator
    E_hat = generator(Z, T)
    H_hat = supervisor(E_hat, T)
    H_hat_supervise = supervisor(H, T)
    
    # Synthetic data
    X_hat = recovery(H_hat, T)
    
    # Discriminator
    Y_fake = discriminator(H_hat, T)
    Y_real = discriminator(H, T)     
    Y_fake_e = discriminator(E_hat, T)
    
    # Variables        
    e_vars = [v for v in tf.compat.v1.trainable_variables() if v.name.startswith('embedder')]
    r_vars = [v for v in tf.compat.v1.trainable_variables() if v.name.startswith('recovery')]
    g_vars = [v for v in tf.compat.v1.trainable_variables() if v.name.startswith('generator')]
    s_vars = [v for v in tf.compat.v1.trainable_variables() if v.name.startswith('supervisor')]
    d_vars = [v for v in tf.compat.v1.trainable_variables() if v.name.startswith('discriminator')]
    
    # Loss for the discriminator
    D_loss_real = tf.compat.v1.losses.sigmoid_cross_entropy(tf.ones_like(Y_real), Y_real)
    D_loss_fake = tf.compat.v1.losses.sigmoid_cross_entropy(tf.zeros_like(Y_fake), Y_fake)
    D_loss_fake_e = tf.compat.v1.losses.sigmoid_cross_entropy(tf.zeros_like(Y_fake_e), Y_fake_e)
    D_loss = D_loss_real + D_loss_fake + gamma * D_loss_fake_e
            
    # Loss for the generator
    # 1. Adversarial loss
    G_loss_U = tf.compat.v1.losses.sigmoid_cross_entropy(tf.ones_like(Y_fake), Y_fake)
    G_loss_U_e = tf.compat.v1.losses.sigmoid_cross_entropy(tf.ones_like(Y_fake_e), Y_fake_e)
    
    # 2. Supervised loss
    G_loss_S = tf.compat.v1.losses.mean_squared_error(H[:,1:,:], H_hat_supervise[:,1:,:])
    
    # 3. Two Momments
    G_loss_V1 = tf.reduce_mean(input_tensor=np.abs(tf.sqrt(tf.nn.moments(x=X_hat,axes=[0])[1] + 1e-6) - tf.sqrt(tf.nn.moments(x=X,axes=[0])[1] + 1e-6)))
    G_loss_V2 = tf.reduce_mean(input_tensor=np.abs((tf.nn.moments(x=X_hat,axes=[0])[0]) - (tf.nn.moments(x=X,axes=[0])[0])))

    G_loss_V = G_loss_V1 + G_loss_V2
    
    # Summation
    G_loss = G_loss_U + gamma * G_loss_U_e + 100 * tf.sqrt(G_loss_S) + 100*G_loss_V 
            
    # Loss for the embedder network
    E_loss_T0 = tf.compat.v1.losses.mean_squared_error(X, X_tilde)
    E_loss0 = 10*tf.sqrt(E_loss_T0)
    E_loss = E_loss0  + 0.1*G_loss_S
    
    # optimizer
    E0_solver = tf.compat.v1.train.AdamOptimizer().minimize(E_loss0, var_list = e_vars + r_vars)
    E_solver = tf.compat.v1.train.AdamOptimizer().minimize(E_loss, var_list = e_vars + r_vars)
    D_solver = tf.compat.v1.train.AdamOptimizer().minimize(D_loss, var_list = d_vars)
    G_solver = tf.compat.v1.train.AdamOptimizer().minimize(G_loss, var_list = g_vars + s_vars)      
    GS_solver = tf.compat.v1.train.AdamOptimizer().minimize(G_loss_S, var_list = g_vars + s_vars)   
        
    ## Sessions    
    
    sess = tf.compat.v1.Session()
    sess.run(tf.compat.v1.global_variables_initializer())
    
    ## Embedding Learning
    
    print('Start Embedding Network Training')
    
    for itt in range(iterations):
        
        # Batch setting
        idx = np.random.permutation(No)
        train_idx = idx[:batch_size]     
            
        X_mb = list(dataX[i] for i in train_idx)
        T_mb = list(dataT[i] for i in train_idx)
            
        # Train embedder        
        _, step_e_loss = sess.run([E0_solver, E_loss_T0], feed_dict={X: X_mb, T: T_mb})
        
        if itt % 1000 == 0:
            print('step: '+ str(itt) + ', e_loss: ' + str(np.round(np.sqrt(step_e_loss),4)) )        
            
    print('Finish Embedding Network Training')
    
    ## Training Supervised Loss First
    
    print('Start Training with Supervised Loss Only')
    
    for itt in range(iterations):
        
        # Batch setting
        idx = np.random.permutation(No)
        train_idx = idx[:batch_size]     
            
        X_mb = list(dataX[i] for i in train_idx)
        T_mb = list(dataT[i] for i in train_idx)        
        
        Z_mb = random_generator(batch_size, z_dim, T_mb, Max_Seq_Len)
        
        # Train generator       
        _, step_g_loss_s = sess.run([GS_solver, G_loss_S], feed_dict={Z: Z_mb, X: X_mb, T: T_mb})
                           
        if itt % 1000 == 0:
            print('step: '+ str(itt) + ', s_loss: ' + str(np.round(np.sqrt(step_g_loss_s),4)) )
                
    print('Finish Training with Supervised Loss Only')
    
    ## Joint Training
    
    print('Start Joint Training')
    
    # Training step
    for itt in range(iterations):
      
        # Generator Training
        for kk in range(2):
          
            # Batch setting
            idx = np.random.permutation(No)
            train_idx = idx[:batch_size]     
            
            X_mb = list(dataX[i] for i in train_idx)
            T_mb = list(dataT[i] for i in train_idx)
            
            # Random vector generation
            Z_mb = random_generator(batch_size, z_dim, T_mb, Max_Seq_Len)
              
            # Train generator
            _, step_g_loss_u, step_g_loss_s, step_g_loss_v = sess.run([G_solver, G_loss_U, G_loss_S, G_loss_V], feed_dict={Z: Z_mb, X: X_mb, T: T_mb})
            
            # Train embedder        
            _, step_e_loss_t0 = sess.run([E_solver, E_loss_T0], feed_dict={Z: Z_mb, X: X_mb, T: T_mb})   
           
        ## Discriminator Training
        
        # Batch setting
        idx = np.random.permutation(No)
        train_idx = idx[:batch_size]     
        
        X_mb = list(dataX[i] for i in train_idx)
        T_mb = list(dataT[i] for i in train_idx)
        
        # Random vector generation
        Z_mb = random_generator(batch_size, z_dim, T_mb, Max_Seq_Len)
            
        
        check_d_loss = sess.run(D_loss, feed_dict={X: X_mb, T: T_mb, Z: Z_mb})
        
        # Train discriminator
        
        if (check_d_loss > 0.15):        
            _, step_d_loss = sess.run([D_solver, D_loss], feed_dict={X: X_mb, T: T_mb, Z: Z_mb})
        
        ## Checkpoints
        if itt % 1000 == 0:
            print('step: '+ str(itt) + 
                  ', d_loss: ' + str(np.round(step_d_loss,4)) + 
                  ', g_loss_u: ' + str(np.round(step_g_loss_u,4)) + 
                  ', g_loss_s: ' + str(np.round(np.sqrt(step_g_loss_s),4)) + 
                  ', g_loss_v: ' + str(np.round(step_g_loss_v,4)) + 
                  ', e_loss_t0: ' + str(np.round(np.sqrt(step_e_loss_t0),4))  )
   
    
    print('Finish Joint Training')
    
    ## Final Outputs
    
    Z_mb = random_generator(No, z_dim, dataT, Max_Seq_Len)
    
    X_hat_curr = sess.run(X_hat, feed_dict={Z: Z_mb, X: dataX, T: dataT})    
    
    ## List of the final outputs
    
    dataX_hat = list()
    
    for i in range(No):
        Temp = X_hat_curr[i,:dataT[i],:]
        dataX_hat.append(Temp)
        
    # Renormalization
    if (Normalization_Flag == 1):
        dataX_hat = dataX_hat * max_val
        dataX_hat = dataX_hat + min_val
    
    return dataX_hat

In [ ]:
## Post-hoc RNN Classifier 

def discriminative_score_metrics (dataX, dataX_hat):
  
    # Initialization on the Graph
    tf.reset_default_graph()

    # Basic Parameters
    No = len(dataX)
    data_dim = len(dataX[0][0,:])
    
    # Compute Maximum seq length and each seq length
    dataT = list()
    Max_Seq_Len = 0
    for i in range(No):
        Max_Seq_Len = max(Max_Seq_Len, len(dataX[i][:,0]))
        dataT.append(len(dataX[i][:,0]))
     
    # Network Parameters
    hidden_dim = max(int(data_dim/2),1)
    iterations = 2000
    batch_size = 128
    
    ## input place holders
    # Features
    X = tf.placeholder(tf.float32, [None, Max_Seq_Len, data_dim], name = "myinput_x")
    X_hat = tf.placeholder(tf.float32, [None, Max_Seq_Len, data_dim], name = "myinput_x_hat")
    
    # Times
    T = tf.placeholder(tf.int32, [None], name = "myinput_t")
    T_hat = tf.placeholder(tf.int32, [None], name = "myinput_t_hat")
    
    ## builde a RNN classification network 
    
    def discriminator (X, T):
      
        with tf.variable_scope("discriminator", reuse = tf.AUTO_REUSE) as vs:
            
            d_cell = tf.nn.rnn_cell.GRUCell(num_units=hidden_dim, activation=tf.nn.tanh, name = 'cd_cell')
                    
            d_outputs, d_last_states = tf.nn.dynamic_rnn(d_cell, X, dtype=tf.float32, sequence_length = T)
                
            # Logits
            Y_hat = tf.contrib.layers.fully_connected(d_last_states, 1, activation_fn=None) 
            
            # Sigmoid output
            Y_hat_Final = tf.nn.sigmoid(Y_hat)
            
            # Variables
            d_vars = [v for v in tf.all_variables() if v.name.startswith(vs.name)]
    
        return Y_hat, Y_hat_Final, d_vars
    
    ## Train / Test Division
    def train_test_divide (dataX, dataX_hat, dataT):
      
        # Divide train/test index
        No = len(dataX)
        idx = np.random.permutation(No)
        train_idx = idx[:int(No*0.8)]
        test_idx = idx[int(No*0.8):]
        
        # Train and Test X
        trainX = [dataX[i] for i in train_idx]
        trainX_hat = [dataX_hat[i] for i in train_idx]
        
        testX = [dataX[i] for i in test_idx]
        testX_hat = [dataX_hat[i] for i in test_idx]
        
        # Train and Test T
        trainT = [dataT[i] for i in train_idx]
        testT = [dataT[i] for i in test_idx]
      
        return trainX, trainX_hat, testX, testX_hat, trainT, testT
    
    ## Functions
    # Variables
    Y_real, Y_pred_real, d_vars = discriminator(X, T)
    Y_fake, Y_pred_fake, _ = discriminator(X_hat, T_hat)
        
    # Loss for the discriminator
    D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Y_real, labels = tf.ones_like(Y_real)))
    D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Y_fake, labels = tf.zeros_like(Y_fake)))
    D_loss = D_loss_real + D_loss_fake
    
    # optimizer
    D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list = d_vars)
        
    ## Sessions    

    # Start session and initialize
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    # Train / Test Division
    trainX, trainX_hat, testX, testX_hat, trainT, testT = train_test_divide (dataX, dataX_hat, dataT)
    
    # Training step
    for itt in range(iterations):
          
        # Batch setting
        idx = np.random.permutation(len(trainX))
        train_idx = idx[:batch_size]     
            
        X_mb = list(trainX[i] for i in train_idx)
        T_mb = list(trainT[i] for i in train_idx)
        
        # Batch setting
        idx = np.random.permutation(len(trainX_hat))
        train_idx = idx[:batch_size]     
            
        X_hat_mb = list(trainX_hat[i] for i in train_idx)
        T_hat_mb = list(trainT[i] for i in train_idx)
          
        # Train discriminator
        _, step_d_loss = sess.run([D_solver, D_loss], feed_dict={X: X_mb, T: T_mb, X_hat: X_hat_mb, T_hat: T_hat_mb})            
        
        ## Checkpoints
#        if itt % 500 == 0:
#            print("[step: {}] loss - d loss: {}".format(itt, np.round(step_d_loss,4)))
    
    ## Final Outputs (ontTesting set)
    
    Y_pred_real_curr, Y_pred_fake_curr = sess.run([Y_pred_real, Y_pred_fake], feed_dict={X: testX, T: testT, X_hat: testX_hat, T_hat: testT})
    
    Y_pred_final = np.squeeze(np.concatenate((Y_pred_real_curr, Y_pred_fake_curr), axis = 0))
    Y_label_final = np.concatenate((np.ones([len(Y_pred_real_curr),]), np.zeros([len(Y_pred_real_curr),])), axis = 0)
    
    ## Accuracy
    Acc = accuracy_score(Y_label_final, Y_pred_final>0.5)
    
    Disc_Score = np.abs(0.5-Acc)
    
    return Disc_Score

# TimeGANs Evaluation

In [ ]:
## PCA Analysis
    
def PCA_Analysis (dataX, dataX_hat):

    # Analysis Data Size
    Sample_No = 1000
    
    # Data Preprocessing
    for i in range(Sample_No):
        if (i == 0):
            arrayX = np.reshape(np.mean(np.asarray(dataX[0]),1), [1,len(dataX[0][:,0])])
            arrayX_hat = np.reshape(np.mean(np.asarray(dataX_hat[0]),1), [1,len(dataX[0][:,0])])
        else:
            arrayX = np.concatenate((arrayX, np.reshape(np.mean(np.asarray(dataX[i]),1), [1,len(dataX[0][:,0])])))
            arrayX_hat = np.concatenate((arrayX_hat, np.reshape(np.mean(np.asarray(dataX_hat[i]),1), [1,len(dataX[0][:,0])])))
    
    # Parameters        
    No = len(arrayX[:,0])
    colors = ["red" for i in range(No)] +  ["blue" for i in range(No)]    
    
    # PCA Analysis
    pca = PCA(n_components = 2)
    pca.fit(arrayX)
    pca_results = pca.transform(arrayX)
    pca_hat_results = pca.transform(arrayX_hat)
        
    # Plotting
    f, ax = plt.subplots(1)
    
    plt.scatter(pca_results[:,0], pca_results[:,1], c = colors[:No], alpha = 0.2, label = "Original")
    plt.scatter(pca_hat_results[:,0], pca_hat_results[:,1], c = colors[No:], alpha = 0.2, label = "Synthetic")

    ax.legend()
    
    plt.title('PCA plot')
    plt.xlabel('x-pca')
    plt.ylabel('y_pca')
    plt.show()
    
    
## TSNE Analysis
    
def tSNE_Analysis (dataX, dataX_hat):
  
    # Analysis Data Size
    Sample_No = 1000
  
    # Preprocess
    for i in range(Sample_No):
        if (i == 0):
            arrayX = np.reshape(np.mean(np.asarray(dataX[0]),1), [1,len(dataX[0][:,0])])
            arrayX_hat = np.reshape(np.mean(np.asarray(dataX_hat[0]),1), [1,len(dataX[0][:,0])])
        else:
            arrayX = np.concatenate((arrayX, np.reshape(np.mean(np.asarray(dataX[i]),1), [1,len(dataX[0][:,0])])))
            arrayX_hat = np.concatenate((arrayX_hat, np.reshape(np.mean(np.asarray(dataX_hat[i]),1), [1,len(dataX[0][:,0])])))
     
    # Do t-SNE Analysis together       
    final_arrayX = np.concatenate((arrayX, arrayX_hat), axis = 0)
    
    # Parameters
    No = len(arrayX[:,0])
    colors = ["red" for i in range(No)] +  ["blue" for i in range(No)]    
    
    # TSNE anlaysis
    tsne = TSNE(n_components = 2, verbose = 1, perplexity = 40, n_iter = 300)
    tsne_results = tsne.fit_transform(final_arrayX)
    
    # Plotting
    f, ax = plt.subplots(1)
    
    plt.scatter(tsne_results[:No,0], tsne_results[:No,1], c = colors[:No], alpha = 0.2, label = "Original")
    plt.scatter(tsne_results[No:,0], tsne_results[No:,1], c = colors[No:], alpha = 0.2, label = "Synthetic")

    ax.legend()
    
    plt.title('t-SNE plot')
    plt.xlabel('x-tsne')
    plt.ylabel('y_tsne')
    plt.show()

In [ ]:
## Post-hoc RNN one-step ahead predictor

def predictive_score_metrics (dataX, dataX_hat):
  
    # Initialization on the Graph
    tf.reset_default_graph()

    # Basic Parameters
    No = len(dataX)
    data_dim = len(dataX[0][0,:])
    
    # Maximum seq length and each seq length
    dataT = list()
    Max_Seq_Len = 0
    for i in range(No):
        Max_Seq_Len = max(Max_Seq_Len, len(dataX[i][:,0]))
        dataT.append(len(dataX[i][:,0]))
     
    # Network Parameters
    hidden_dim = max(int(data_dim/2),1)
    iterations = 5000
    batch_size = 128
    
    ## input place holders
    
    X = tf.placeholder(tf.float32, [None, Max_Seq_Len-1, data_dim-1], name = "myinput_x")
    T = tf.placeholder(tf.int32, [None], name = "myinput_t")    
    Y = tf.placeholder(tf.float32, [None, Max_Seq_Len-1, 1], name = "myinput_y")
    
    ## builde a RNN discriminator network 
    
    def predictor (X, T):
      
        with tf.variable_scope("predictor", reuse = tf.AUTO_REUSE) as vs:
            
            d_cell = tf.nn.rnn_cell.GRUCell(num_units=hidden_dim, activation=tf.nn.tanh, name = 'd_cell')
                    
            d_outputs, d_last_states = tf.nn.dynamic_rnn(d_cell, X, dtype=tf.float32, sequence_length = T)
                
            Y_hat = tf.contrib.layers.fully_connected(d_outputs, 1, activation_fn=None) 
            
            Y_hat_Final = tf.nn.sigmoid(Y_hat)
            
            d_vars = [v for v in tf.all_variables() if v.name.startswith(vs.name)]
    
        return Y_hat_Final, d_vars
    
    ## Functions
    # Variables
    Y_pred, d_vars = predictor(X, T)
        
    # Loss for the predictor
    D_loss = tf.losses.absolute_difference(Y, Y_pred)
    
    # optimizer
    D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list = d_vars)
        
    ## Sessions    

    # Session start
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    # Training using Synthetic dataset
    for itt in range(iterations):
          
        # Batch setting
        idx = np.random.permutation(len(dataX_hat))
        train_idx = idx[:batch_size]     
            
        X_mb = list(dataX_hat[i][:-1,:(data_dim-1)] for i in train_idx)
        T_mb = list(dataT[i]-1 for i in train_idx)
        Y_mb = list(np.reshape(dataX_hat[i][1:,(data_dim-1)],[len(dataX_hat[i][1:,(data_dim-1)]),1]) for i in train_idx)        
          
        # Train discriminator
        _, step_d_loss = sess.run([D_solver, D_loss], feed_dict={X: X_mb, T: T_mb, Y: Y_mb})            
        
        ## Checkpoints
#        if itt % 500 == 0:
#            print("[step: {}] loss - d loss: {}".format(itt, np.sqrt(np.round(step_d_loss,4))))
    
    ## Use Original Dataset to test
    
    # Make Batch with Original Data
    idx = np.random.permutation(len(dataX_hat))
    train_idx = idx[:No]     
    
    X_mb = list(dataX[i][:-1,:(data_dim-1)] for i in train_idx)
    T_mb = list(dataT[i]-1 for i in train_idx)
    Y_mb = list(np.reshape(dataX[i][1:,(data_dim-1)], [len(dataX[i][1:,(data_dim-1)]),1]) for i in train_idx)
    
    # Predict Fugure
    pred_Y_curr = sess.run(Y_pred, feed_dict={X: X_mb, T: T_mb})
    
    # Compute MAE
    MAE_Temp = 0
    for i in range(No):
        MAE_Temp = MAE_Temp + mean_absolute_error(Y_mb[i], pred_Y_curr[i,:,:])
    
    MAE = MAE_Temp / No
    
    return MAE

# DISEASE TRAJECTORY STATE PREDICTIONS - MODEL CODE

In [ ]:
# MODEL CLASSES:
class HMM:
    
    def __init__(self, num_states):
        
        self.num_states = num_states
        self.hmm_model  = hmm.GaussianHMM(n_components=num_states)
        
        
    def fit(self, X_obs):
        
        X_obser = [[list(X_obs[u][k,:]) for k in range(len(X_obs[u]))] for u in range(len(X_obs))]
        X_obs_  = np.concatenate(X_obser)

        lengths = [len(X_obs[k]) for k in range(len(X_obs))]
        
        self.hmm_model.fit(X_obs_, lengths)
        
        
    def predict(self, X_new):
        
        preds       = np.concatenate([self.hmm_model.predict(X_new[k]) for k in range(len(X_new))])
        predictions = [np.sum(np.concatenate([(self.hmm_model.transmat_[preds[k],u] * self.hmm_model.means_[u]).reshape((1,-1)) for u in range(self.num_states)]), axis=0).reshape((1,-1)) for k in range(len(preds))] 
        
        return np.concatenate(predictions)
        


@tf_export("nn.rnn_cell.MultiRNNCell")
class MultiPhasedLSTMCell(MultiRNNCell):

    def __init__(self, cells, state_is_tuple=True):
        
        super(MultiRNNCell, self).__init__()
        
        if not cells:
            raise ValueError("Must specify at least one cell for MultiRNNCell.")
        if not nest.is_sequence(cells):
            raise TypeError("cells must be a list or tuple, but saw: %s." % cells)

        self._cells = cells
        for cell_number, cell in enumerate(self._cells):
        
            # Add Checkpointable dependencies on these cells so their variables get
            # saved with this object when using object-based saving.
            if isinstance(cell, checkpointable.CheckpointableBase):
                # TODO(allenl): Track down non-Checkpointable callers.
                self._track_checkpointable(cell, name="cell-%d" % (cell_number,))
                self._state_is_tuple = state_is_tuple
            
            if not state_is_tuple:
                if any(nest.is_sequence(c.state_size) for c in self._cells):
                    raise ValueError("Some cells return tuples of states, but the flag "
                                     "state_is_tuple is not set.  State sizes are: %s"
                                     % str([c.state_size for c in self._cells]))

    @property
    def state_size(self):
        if self._state_is_tuple:
            return tuple(cell.state_size for cell in self._cells)
        else:
            return sum([cell.state_size for cell in self._cells])

    @property
    def output_size(self):
        return self._cells[-1].output_size

    def zero_state(self, batch_size, dtype):
        with ops.name_scope(type(self).__name__ + "ZeroState", values=[batch_size]):
            if self._state_is_tuple:
                return tuple(cell.zero_state(batch_size, dtype) for cell in self._cells)
            else:
                # We know here that state_size of each cell is not a tuple and
                # presumably does not contain TensorArrays or anything else fancy
                return super(MultiRNNCell, self).zero_state(batch_size, dtype)

    def call(self, inputs, state):
        """Run this multi-layer cell on inputs, starting from state."""
        cur_state_pos = 0

        times         = inputs[0]
        cur_inp       = inputs[1]
        
        new_states    = []
    
        for i, cell in enumerate(self._cells):
            with vs.variable_scope("cell_%d" % i):
                if self._state_is_tuple:
                    if not nest.is_sequence(state):
                        raise ValueError("Expected state to be a tuple of length %d, but received: %s" %(len(self.state_size), state))
                    cur_state = state[i]
                else:
                    cur_state = array_ops.slice(state, [0, cur_state_pos],
                                      [-1, cell.state_size])
                    cur_state_pos += cell.state_size
                
                cur_inp, new_state = cell((times, cur_inp), cur_state)
                new_states.append(new_state)

        new_states = (tuple(new_states) if self._state_is_tuple else array_ops.concat(new_states, 1))

        return cur_inp, new_states


def lazy_property(function):
    attribute = '_' + function.__name__

    @property
    @functools.wraps(function)
    def wrapper(self):
        if not hasattr(self, attribute):
            setattr(self, attribute, function(self))
        return getattr(self, attribute)
    return wrapper


def padd_data(X, padd_length):
    
    X_padded      = []
    
    for k in range(len(X)):
        
        if X[k].shape[0] < padd_length:
            
            if len(X[k].shape) > 1:
                X_padded.append(np.array(np.vstack((np.array(X[k]), 
                                                    np.zeros((padd_length-X[k].shape[0],X[k].shape[1]))))))
            else:
                X_padded.append(np.array(np.vstack((np.array(X[k]).reshape((-1,1)),
                                                    np.zeros((padd_length-X[k].shape[0],1))))))
                
        else:
            
            if len(X[k].shape) > 1:
                X_padded.append(np.array(X[k]))
            else:
                X_padded.append(np.array(X[k]).reshape((-1,1)))
  

    X_padded      = np.array(X_padded)

    return X_padded


def flatten_sequences_to_numpy(sequence_list):
    
    seqLists   = [list(itertools.chain.from_iterable(sequence_list[k].tolist())) for k in range(len(sequence_list))]
    flat_seqs  = np.array(list(itertools.chain.from_iterable(seqLists)))
    
    return flat_seqs


def state_to_array(state_index, number_of_states):
    
    state_array = np.zeros(number_of_states)
    state_array[state_index] = 1
    
    return state_array
    

def get_transitions(state_array, num_states):
    
    trans_matrix = np.zeros((num_states, num_states))
    each_state   = [np.sum((state_array==k)*1) for k in range(num_states)]
    
    for k in range(num_states):
        
        where_states       = np.where(state_array==k)[0]
        where_states_      = where_states[where_states < len(state_array) - 1]
        
        after_states       = [state_array[where_states_[k] + 1] for k in range(len(where_states_))]
        trans_matrix[k, :] = np.array([(np.where(np.array(after_states)==k))[0].shape[0] for k in range(num_states)])
        
    return trans_matrix, each_state


class attentive_state_space_model:
    
    '''
    Class for the "Attentive state space model" implementation. Based on the paper: 
    "Attentive state space model for disease progression" by Ahmed M. Alaa and Mihaela van der Schaar.
    
    ** Key arguments **
    
    :param maximum_seq_length: Maximum allowable length for any trajectory in the training data. 
    :param input_dim: Dimensionality of the observations (emissions).
    :param num_states: Cardinality of the state space.
    :param inference_network: Configuration of the inference network. Default is: 'Seq2SeqAttention'.
    :param rnn_type: Type of RNN cells to use in the inference network. Default is 'LSTM'.
    :param unsupervised: Boolean for whether the model is supervised or unsupervised. Default is True. 
                         Supervised is NOT IMPLEMENTED.
    :param generative: Boolean for whether to enable sampling from the model. 
    :param irregular: Whether the trajectories are in continuous time. NOT IMPLEMENTED.
    :param multitask: Boolean for whether multi-task output layers are used in inference network. NOT IMPLEMENTED
    :param num_iterations: Number of iterations for the stochastic variational inference algorithm.
    :param num_epochs: Number of epochs for the stochastic variational inference algorithm.
    :param batch_size: Size of the batch subsampled from the training data.
    :param learning_rate: Learning rate for the ADAM optimizer. (TO DO: enable selection of the optimizer)
    :param num_rnn_hidden: Size of the RNN layers used in the inference network.
    :param num_rnn_layers: Number of RNN layers used in the inference network.
    :param dropout_keep_prob: Dropout probability. Default is None.
    :param num_out_hidden: Size of output layer in inference network.
    :param num_out_layers: Size of output layer in inference network.
    ** Key attributes **
    
    After fitting the model, the key model parameters are stored in the following attributes:
    
    :attr states_mean: Mean of each observation in each of the num_states states.
    :attr states_covars: Covariance matrices of observations.
    :attr transition_matrix: Baseline Markov transition matrix for the attentive state space.
    :attr intial probabilities: Initial distribution of states averaged accross all trajectories in training data.
    
    ** Key methods **
    
    Three key methods are implemented in the API:
    
    :method fit: Takes a list of observations and fits an attentive state space model in an unsupervised fashion.
    :method predict: Takes a new observation and returns three variables:
                     - Prediction of the next state at every time step.
                     - Expected observation at the next time tep.
                     - List of attention weights assigned to previous states at every time step.
    :method sample: This method samples synthetic trajectories from the model.
    '''
    
    def __init__(self, 
                 maximum_seq_length, 
                 input_dim, 
                 num_states=3,
                 inference_network='Seq2SeqAttention', 
                 rnn_type='LSTM',
                 unsupervised=True,
                 generative=True,
                 irregular=False,
                 multitask=False,
                 input_name="Input", 
                 output_name="Output",
                 model_name="SeqModel",
                 num_iterations=50, 
                 num_epochs=10, 
                 batch_size=100, 
                 learning_rate=5*1e-4, 
                 num_rnn_hidden=100, 
                 num_rnn_layers=1,
                 dropout_keep_prob=None,
                 num_out_hidden=100, 
                 num_out_layers=1,
                 verbosity=True,
                 **kwargs
                ):
        
        # Set all model variables

        self.maximum_seq_length = maximum_seq_length 
        self.input_dim          = input_dim
        self.num_states         = num_states
        self.inference_network  = inference_network
        self.rnn_type           = rnn_type
        self.unsupervised       = unsupervised
        self.generative         = generative
        self.irregular          = irregular
        self.multitask          = multitask
        self.input_name         = input_name 
        self.output_name        = output_name 
        self.model_name         = model_name
        self.num_iterations     = num_iterations
        self.num_epochs         = num_epochs
        self.batch_size         = batch_size
        self.learning_rate      = learning_rate
        self.num_rnn_hidden     = num_rnn_hidden
        self.num_rnn_layers     = num_rnn_layers
        self.dropout_keep_prob  = dropout_keep_prob
        self.num_out_hidden     = num_out_hidden
        self.num_out_layers     = num_out_layers
        self.verbosity          = verbosity
        
        
        self.build_attentive_inference_network()
        
        tf.reset_default_graph()
        
        self.build_attentive_inference_graph()
        
        
    
    def build_attentive_inference_network(self):
        
        # replace this with dictionary style indexing
        
        model_options_names     = ['RNN','LSTM','GRU','PhasedLSTM']
        
        optimizer_options_names = []
        
        
        model_options   = [BasicRNNCell(self.num_rnn_hidden), 
                           rnn_cell.LSTMCell(self.num_rnn_hidden), 
                           rnn_cell.GRUCell(self.num_rnn_hidden), 
                           PhasedLSTMCell(self.num_rnn_hidden)]
        
        self._rnn_model = model_options[np.where(np.array(model_options_names)==self.rnn_type)[0][0]]
        
        if self.dropout_keep_prob is not None:
            
            self._rnn_model = tf.nn.rnn_cell.DropoutWrapper(self._rnn_model, output_keep_prob=self.dropout_keep_prob)
        
        self._Losses = []
        

    def build_attentive_inference_graph(self):
        
        self.observation = tf.placeholder(tf.float32, 
                                     [None, self.maximum_seq_length, self.input_dim], 
                                     name=self.input_name)
            
        self.state_guess = tf.placeholder(tf.float32, 
                                     [None, self.maximum_seq_length, self.num_states]) 
        
        if self.irregular:
            
            self.times      = tf.placeholder(tf.float32, [None, self.maximum_seq_length, 1])
            self.rnn_input  = (self.times, self.observation)
        
        else:
            
            self.rnn_input  = self.observation

            
    @lazy_property
    def length(self):
        
        used   = tf.sign(tf.reduce_max(tf.abs(self.observation), reduction_indices=2))
        length = tf.reduce_sum(used, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        
        return length

    @lazy_property
    def forward(self):
        
        self.attentive_inference_network_inputs()
        
        # Recurrent network.   
        if self.inference_network != 'Seq2SeqAttention': 
            
            rnn_output, _  = rnn.dynamic_rnn(self._rnn_model, 
                                             self.rnn_input_, 
                                             dtype=tf.float32, 
                                             sequence_length=self.length_,)
        else:
            
            try:
                
                tf.nn.seq2seq = tf.contrib.legacy_seq2seq
                tf.nn.rnn_cell = tf.contrib.rnn
                tf.nn.rnn_cell.GRUCell = tf.contrib.rnn.GRUCell

                if self.verbosity:
                    
                    print("TensorFlow version : >= 1.0")
            
            except: 
            
                print("TensorFlow version : 0.12")
            
            if self.verbosity:
                
                print("---------------------------")
            
            self.enc_inp    = [self.rnn_input_[:, t, :] for t in range(self.maximum_seq_length)]

            self.dec_output = [tf.placeholder(tf.float32, shape=(None, 1), 
                                              name="dec_output_".format(t)) for t in range(self.maximum_seq_length)]

            self.dec_inp    = [tf.zeros_like(self.enc_inp[0], dtype=np.float32, name="GO")] + self.enc_inp[:-1] 

            self.cells = []
    
            for i in range(self.num_rnn_layers):
                
                with tf.variable_scope('RNN_{}'.format(i)):
                    
                    self.cells.append(tf.nn.rnn_cell.LSTMCell(self.num_rnn_hidden))
            
            
            self.cell  = tf.nn.rnn_cell.MultiRNNCell(self.cells)
            self.dec_outputs, self.dec_memory = tf.nn.seq2seq.basic_rnn_seq2seq(self.enc_inp, self.dec_inp, self.cell) 
            
            self.weight_dec, self.bias_dec = self._weight_and_bias(self.num_rnn_hidden, 1, ["w_dec", "b_dec"])
            
            self.seq2seq_attn = [(tf.matmul(i, self.weight_dec) + self.bias_dec) for i in self.dec_outputs]
            self.seq2seq_attn = tf.nn.softmax(tf.reshape(tf.stack(self.seq2seq_attn), 
                                                         [-1, self.maximum_seq_length, 1]), axis=1)
            
        
        # Softmax layer.
        self.combiner_func_weight, self.combiner_func_bias = self._weight_and_bias(self.input_dim, 
                                                                                   self.num_out_hidden, 
                                                                                   ["w_0", "b_0"])
        
        self.weight, self.bias     = self._weight_and_bias(self.num_out_hidden, 
                                                           self.num_states, 
                                                           ["w", "b"])
            
        # Flatten to apply same weights to all time steps.
        
        if self.inference_network not in ['RETAIN', 'Seq2SeqAttention']: 
            
            rnn_output  = tf.reshape(rnn_output, [-1, self.num_out_hidden])
            
            forward     = tf.nn.softmax(tf.matmul(rnn_output, self.weight) + self.bias)
        
        elif self.inference_network == 'RETAIN':
            
            self.weight_a, self.bias_a = self._weight_and_bias(self.num_out_hidden, 1, ["w_a", "b_a"])
            
            rnn_output      = tf.reshape(rnn_output, [-1, self.num_out_hidden])
            
            self.attention  = tf.nn.softmax(tf.reshape(tf.matmul(rnn_output, self.weight_a) + self.bias_a, 
                                                       [-1, self.maximum_seq_length, 1]), axis=1)
            

            attn_mask       = tf.expand_dims(tf.sign(tf.reduce_max(tf.abs(self.rnn_input_), reduction_indices=2)), axis=2)
            masked_attn     = tf.multiply(attn_mask, self.attention)
            attn_norms      = tf.expand_dims(tf.tile(tf.reduce_sum(masked_attn, axis=1), [1, self.maximum_seq_length]), axis=2)
            self.attention  = masked_attn/attn_norms
            self.attention_ = tf.tile(self.attention, [1, 1, self.input_dim])
            self.context    = tf.reduce_sum(tf.multiply(self.attention_, self.rnn_input_), reduction_indices=1)
            
            context_layer   = tf.matmul(self.context, self.combiner_func_weight ) + self.combiner_func_bias
            forward         = tf.nn.softmax(tf.matmul(context_layer, self.weight) + self.bias)
        
        elif self.inference_network == 'Seq2SeqAttention':
            
            self.attention  = self.seq2seq_attn
            
            attn_mask       = tf.expand_dims(tf.sign(tf.reduce_max(tf.abs(self.rnn_input_), reduction_indices=2)), axis=2)            
            masked_attn     = tf.multiply(attn_mask, self.attention)
            attn_norms      = tf.expand_dims(tf.tile(tf.reduce_sum(masked_attn, axis=1), [1, self.maximum_seq_length]), axis=2)
            self.attention  = masked_attn/attn_norms
            self.attention_ = tf.tile(self.attention, [1, 1, self.input_dim])
            self.context    = tf.reduce_sum(tf.multiply(self.attention_, self.rnn_input_), reduction_indices=1)
            context_layer   = tf.matmul(self.context, self.combiner_func_weight ) + self.combiner_func_bias
            forward         = tf.nn.softmax(tf.matmul(context_layer, self.weight) + self.bias)

        forward         = tf.reshape(forward, [-1, self.maximum_seq_length, self.num_states])
        
        self.predicted  = forward
        self.predicted  = tf.identity(self.predicted, name=self.output_name)
        
        return forward

    
    def attentive_inference_network_inputs(self):
        
        if self.inference_network in ['RETAIN', 'Seq2SeqAttention']: 
            
            self.num_samples = tf.shape(self.observation)[0] 
             
            Lengths_         = np.repeat(self.length, self.maximum_seq_length)
            
            conv_data        = tf.reshape(tf.tile(self.observation, [1, self.maximum_seq_length, 1]), 
                                          [self.maximum_seq_length * self.num_samples, 
                                           self.maximum_seq_length, self.input_dim])
            
            conv_mask_       = tf.ones([self.maximum_seq_length, self.maximum_seq_length], tf.float32)
            
            conv_mask        = tf.tile(tf.expand_dims(tf.tile(tf.matrix_band_part(conv_mask_, -1, 0), 
                                                              [self.num_samples, 1]), 2), 
                                                              [1, 1, self.input_dim])
            
            masked_data   = tf.multiply(conv_data, conv_mask)
            
            Seq_lengths_  = tf.tile(tf.range(1, self.maximum_seq_length + 1, 1), [self.num_samples])
            
            if self.inference_network == 'RETAIN':
            
                self.rnn_input_  = tf.reverse_sequence(masked_data, batch_axis=0, seq_dim=1, 
                                                       seq_lengths=Seq_lengths_, seq_axis=None)
            else:
                
                self.rnn_input_  = masked_data
                
            
            used         = tf.sign(tf.reduce_max(tf.abs(self.rnn_input_), reduction_indices=2))
            length       = tf.reduce_sum(used, reduction_indices=1)
            self.length_ = tf.cast(length, tf.int32)

            
        else:    
            
            self.rnn_input_ = self.rnn_input
            self.length_    = self.length

            
    @lazy_property
    def ELBO(self):

        mask                = tf.sign(tf.reduce_max(tf.abs(self.observation), reduction_indices=2))
        flat_mask           = tf.reshape(mask, [-1,1])
        
        flat_state_guess    = tf.reshape(self.state_guess, [-1, self.num_states])      
        flat_forward        = tf.reshape(self.forward, [-1, self.num_states])

        likelihood_loss     = tf.reduce_sum(-1*(flat_state_guess * tf.log(flat_forward)))
        
        self.mask           = mask
        
        # Average over actual sequence lengths. << Did I forget masking padded ELBOs? >>
        likelihood_loss     /= tf.reduce_sum(tf.cast(self.length, tf.float32),reduction_indices=0)
        
        return likelihood_loss        
            
        
    def initialize_hidden_states(self, X):
        
        self.init_states = GaussianMixture(n_components=self.num_states, 
                                           covariance_type='full')
        # print('Dims: ', self.input_dim)
        # print('X: ', X[0:2])
        self.init_states.fit(np.concatenate(X).reshape((-1, self.input_dim))) 
        
        
    def get_likelihood(self, X, pred):
        
        likelihoods_ = []
 
        XX    = X.reshape((-1, self.input_dim))
        
        # lks_  = np.array([multivariate_normal.logpdf(XX, self.state_means[k], self.state_covars[k]).reshape((-1,1))*pred[:,k].reshape((-1,1)) for k in range(self.num_states)])
        # TODO: to get to work with more features, added allow_singular=True, come back to see if better fix
        lks_  = np.array([multivariate_normal.logpdf(XX, self.state_means[k], self.state_covars[k], allow_singular=True).reshape((-1,1))*pred[:,k].reshape((-1,1)) for k in range(self.num_states)])

        likelihoods_  = lks_
        
        return np.mean(likelihoods_[np.isfinite(likelihoods_)])
    
    
    def sample_posterior_states(self, q_posterior):
        
        sampled_list = [state_to_array(np.random.choice(self.num_states, 1, p=q_posterior[k,:])[0], self.num_states) for k in range(q_posterior.shape[0])]
        self.state_trajectories = np.array(sampled_list)
    
    def fit(self, X, T=None):
        
        self.state_trajectories_ = []
        
        self.initialize_hidden_states(X)

        state_inferences_init  = [np.argmax(self.init_states.predict_proba(X[k]), axis=1) for k in range(len(X))]
        self.all_states        = state_inferences_init

        for v in range(len(state_inferences_init)):
            
            state_list = [state_to_array(state_inferences_init[v][k], self.num_states) for k in range(len(state_inferences_init[v]))]
            delayed_traject = np.vstack((np.array(state_list)[1:, :], np.array(state_list)[-1, :]))
            
            self.state_trajectories_.append(delayed_traject)
            
            
        self.normalizer   = StandardScaler()
        self.normalizer.fit(np.concatenate(X))

        self.X_normalized  = []

        for k in range(len(X)):
            
            self.X_normalized.append(self.normalizer.transform(X[k])) 
    

        self.stochastic_variational_inference(self.X_normalized)

        
    def stochastic_variational_inference(self, X, T=None):
        
        X_, state_update = padd_data(X, self.maximum_seq_length), padd_data(self.state_trajectories_, self.maximum_seq_length)
        
        if T is not None:
            T_   = padd_data(T, self.maximum_seq_length)
            
            
        # Baseline transition matrix

        initial_states = np.array([self.all_states[k][0] for k in range(len(self.all_states))])
        init_probs     = [np.where(initial_states==k)[0].shape[0] / len(initial_states) for k in range(self.num_states)]

        transits   = np.zeros((self.num_states, self.num_states))
        each_state = np.zeros(self.num_states)

        for _ in range(len(self.all_states)):
    
            new_trans, new_each_state = get_transitions(self.all_states[_], self.num_states)
    
            transits   += new_trans
            each_state += new_each_state
    
        for _ in range(self.num_states):
    
            transits[_, :] = transits[_, :] / each_state[_]
            transits[_, :] = transits[_, :] / np.sum(transits[_, :])
    
        self.initial_probabilities = np.array(init_probs)
        self.transition_matrix     = np.array(transits)
        
        # -----------------------------------------------------------
        # Observational distribution
        # -----------------------------------------------------------
        
        self.state_means  = self.init_states.means_
        self.state_covars = self.init_states.covariances_    
        
        
        sess = tf.InteractiveSession()
        
        opt      = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.ELBO)
        init     = tf.global_variables_initializer()
        
        sess.run(init)

        saver = tf.train.Saver()
        
        for epoch in range(self.num_epochs):
                
            for _ in range(self.num_iterations):
                
                batch_samples = np.random.choice(list(range(X_.shape[0])), size=self.batch_size, replace=False)
                # print(batch_samples[0:3])
                # print('batch samples: ', batch_samples[0:10])
                # print(X_)
                # print('x shape 1: ', X_.shape)
                # # X_ = X_.reshape(X_.shape[0],3)
                # X_ = X_.reshape(int(X_.shape[0]/3),3)
                # print('x shape 2: ', X_.shape)
                
                batch_train   = X_[batch_samples,:,:]
                # batch_train   = X_[batch_samples , : , : ]
                batch_states  = state_update[batch_samples,:,:]
                
                train_dict    = {self.observation : batch_train,
                                 self.state_guess : batch_states}
                
                batch_preds   = sess.run(self.forward, feed_dict=train_dict)
                
                # sample and update posterior states
                self.sample_posterior_states(batch_preds.reshape((-1, self.num_states)))

                sess.run(opt, feed_dict=train_dict)
                
                Loss = sess.run(self.ELBO, feed_dict=train_dict)
                
                log_likelihood_ = np.array([self.get_likelihood(batch_train[k,:,:], batch_preds[k,:,:]) for k in range(batch_train.shape[0])]) 
                log_likelihood_ = np.sum(log_likelihood_)/self.batch_size
                
                self._Losses.append(log_likelihood_)
 
                # Verbosity function
    
                if self.verbosity:
            
                    print('Epoch %d \t----- \tBatch %d \t----- \tLog-Likelihood %10.6e' % (epoch, _, log_likelihood_))
                
        
        # Save model
        saver.save(sess, "./mlaimRNN_model") 
        
        if os.path.exists("attentive_state_space"):
            
            shutil.rmtree("attentive_state_space")
        
        tf.saved_model.simple_save(sess, export_dir='attentive_state_space', inputs={"myInput": self.observation}, 
                                   outputs={"myOutput": self.predicted})    
        
           

    def predict(self, X):
        
        X_normalized  = []

        for k in range(len(X)):
            
            X_normalized.append(self.normalizer.transform(X[k])) 
        
        with tf.Session() as sess:
            
            saver           = tf.train.import_meta_graph("mlaimRNN_model.meta")
            saver.restore(sess, tf.train.latest_checkpoint('./'))
            
            preds_lengths   = [len(X_normalized[k]) for k in range(len(X_normalized))]
            
            X_pred          = padd_data(X_normalized, padd_length=self.maximum_seq_length)
            pred_dict       = {self.observation  : X_pred}
            
            prediction_     = sess.run(self.forward, pred_dict).reshape([-1, self.maximum_seq_length, self.num_states])         

            preds_          = []
            obs_            = []
            
            for k in range(len(X)):
                
                preds_.append(prediction_[k, 0 : preds_lengths[k]])
                obs_.append(self.get_observations(preds_[-1]))

            if self.inference_network in ['RETAIN', 'Seq2SeqAttention']:
                
                
                attn_                  = sess.run(self.attention, pred_dict) 
                attn_per_patient       = [attn_[u * self.maximum_seq_length : u * self.maximum_seq_length + self.maximum_seq_length, :, :] for u in range(len(X))]
                attn_lists_per_patient = [[attn_per_patient[u][k, 0 : k + 1, :] for k in range(self.maximum_seq_length)] for u in range(len(X))]
                
                all_preds_             = (preds_, obs_, attn_lists_per_patient)    

            else:
                
                all_preds_             = (preds_, obs_) 

                
        return all_preds_ 
    
    
    def get_observations(self, preds):
        
        pred_obs     = []
    
        for v in range(preds.shape[0]):
        
            observations = np.zeros(self.input_dim)
    
            for k in range(self.num_states):
            
                observations += self.state_means[k] * preds[v, k] 
    
            pred_obs.append(observations) 
    
    
        return np.array(pred_obs)
    
    
    def sample(self, trajectory_length=5):
        
        initial_state    = np.random.choice(self.num_states, 1, 
                                            p=self.initial_probabilities)[0]
    
        State_trajectory      = [initial_state]
        first_observation     = np.random.multivariate_normal(self.state_means[initial_state], 
                                                              self.state_covars[initial_state]) 
    
        Obervation_trajectory = [first_observation.reshape((1,-1))]
    
        for _ in range(trajectory_length):
        
            next_state_pred  = self.predict(Obervation_trajectory)[0][0][0]
            next_state       = np.random.choice(self.num_states, 1, p=next_state_pred)[0]
        
            State_trajectory.append(next_state)
        
            next_observation = np.random.multivariate_normal(self.state_means[next_state], 
                                                         self.state_covars[next_state]).reshape((1,-1))
        
        
            Obervation_trajectory[0] = np.vstack((Obervation_trajectory[0], next_observation))
        
    
        return State_trajectory, Obervation_trajectory   
    
    
    @staticmethod
    def _weight_and_bias(in_size, out_size, wnames):
    
        weight = tf.get_variable(wnames[0], 
                                 shape=[in_size, out_size], 
                                 initializer=tf.contrib.layers.xavier_initializer())
        
        bias   = tf.get_variable(wnames[1], 
                                 shape=[out_size], 
                                 initializer=tf.contrib.layers.xavier_initializer())
        
        return weight, bias
        



class SeqModel:
    
    '''
    Parent class for all RNN models.
    '''
    
    def __init__(self, 
                 maximum_seq_length, 
                 input_dim, 
                 output_dim=1,
                 model_type='RNN',
                 rnn_type='RNN',
                 latent=False,
                 generative=False,
                 irregular=False,
                 multitask=False,
                 prediction_mode='Sequence_labeling',
                 input_name="Input", 
                 output_name="Output",
                 model_name="SeqModel",
                 num_iterations=20, 
                 num_epochs=10, 
                 batch_size=100, 
                 learning_rate=0.0005, 
                 num_rnn_hidden=200, 
                 num_rnn_layers=1,
                 dropout_keep_prob=None,
                 num_out_hidden=200, 
                 num_out_layers=1,
                 **kwargs
                ):
        
        # Set all model variables

        self.maximum_seq_length = maximum_seq_length 
        self.input_dim          = input_dim
        self.output_dim         = output_dim
        self.model_type         = model_type
        self.rnn_type           = rnn_type
        self.latent             = latent
        self.generative         = generative
        self.irregular          = irregular
        self.multitask          = multitask
        self.prediction_mode    = prediction_mode
        self.input_name         = input_name 
        self.output_name        = output_name 
        self.model_name         = model_name
        self.num_iterations     = num_iterations
        self.num_epochs         = num_epochs
        self.batch_size         = batch_size
        self.learning_rate      = learning_rate
        self.num_rnn_hidden     = num_rnn_hidden
        self.num_rnn_layers     = num_rnn_layers
        self.dropout_keep_prob  = dropout_keep_prob
        self.num_out_hidden     = num_out_hidden
        self.num_out_layers     = num_out_layers
        
        
        self.build_rnn_model()
        tf.reset_default_graph()
        self.build_rnn_graph()
        
        
    
    def build_rnn_model(self):
        
        # replace this with dictionary style indexing
        
        model_options_names     = ['RNN','LSTM','GRU','PhasedLSTM']
        
        optimizer_options_names = []
        
        
        model_options   = [BasicRNNCell(self.num_rnn_hidden), rnn_cell.LSTMCell(self.num_rnn_hidden), 
                                   rnn_cell.GRUCell(self.num_rnn_hidden), PhasedLSTMCell(self.num_rnn_hidden)]
        
        self._rnn_model = model_options[np.where(np.array(model_options_names)==self.rnn_type)[0][0]]
        
        if self.dropout_keep_prob is not None:
            
            self._rnn_model = tf.nn.rnn_cell.DropoutWrapper(self._rnn_model, output_keep_prob=self.dropout_keep_prob)
        
        self._Losses = []
        

    def build_rnn_graph(self):
        
        self.data   = tf.placeholder(tf.float32, 
                                     [None, self.maximum_seq_length, self.input_dim], 
                                     name=self.input_name)
            
        self.target = tf.placeholder(tf.float32, 
                                     [None, self.maximum_seq_length, self.output_dim]) 
        
        if self.irregular:
            
            self.times      = tf.placeholder(tf.float32, [None, self.maximum_seq_length, 1])
            self.rnn_input  = (self.times, self.data)
        
        else:
            
            self.rnn_input  = self.data 

            
    @lazy_property
    def length(self):
        
        used   = tf.sign(tf.reduce_max(tf.abs(self.data), reduction_indices=2))
        length = tf.reduce_sum(used, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        
        return length

    @lazy_property
    def prediction(self):
        
        self.process_rnn_inputs()
        
        # Recurrent network.   
        if self.model_type != 'Seq2SeqAttention': 
            
            rnn_output, _  = rnn.dynamic_rnn(self._rnn_model, 
                                             self.rnn_input_, 
                                             dtype=tf.float32, 
                                             sequence_length=self.length_,)
        else:
            
            try:
                
                tf.nn.seq2seq = tf.contrib.legacy_seq2seq
                tf.nn.rnn_cell = tf.contrib.rnn
                tf.nn.rnn_cell.GRUCell = tf.contrib.rnn.GRUCell
                print("TensorFlow version : >= 1.0")
            
            except: 
            
                print("TensorFlow version : 0.12")
            
            self.enc_inp    = [self.rnn_input_[:, t, :] for t in range(self.maximum_seq_length)]

            self.dec_output = [tf.placeholder(tf.float32, shape=(None, 1), 
                                              name="dec_output_".format(t)) for t in range(self.maximum_seq_length)]

            self.dec_inp    = [tf.zeros_like(self.enc_inp[0], dtype=np.float32, name="GO")] + self.enc_inp[:-1] 

            self.cells = []
    
            for i in range(self.num_rnn_layers):
                
                with tf.variable_scope('RNN_{}'.format(i)):
                    
                    self.cells.append(tf.nn.rnn_cell.GRUCell(self.num_rnn_hidden))
            
            
            self.cell  = tf.nn.rnn_cell.MultiRNNCell(self.cells)
            self.dec_outputs, self.dec_memory = tf.nn.seq2seq.basic_rnn_seq2seq(self.enc_inp, self.dec_inp, self.cell) 
            
            self.weight_dec, self.bias_dec = self._weight_and_bias(self.num_rnn_hidden, self.output_dim, ["w_dec", "b_dec"])
            
            self.seq2seq_attn = [(tf.matmul(i, self.weight_dec) + self.bias_dec) for i in self.dec_outputs]
            self.seq2seq_attn = tf.nn.softmax(tf.reshape(tf.stack(self.seq2seq_attn), 
                                                         [-1, self.maximum_seq_length, 1]), axis=1)
            
        
        # Softmax layer.
        self.weight_0, self.bias_0 = self._weight_and_bias(self.input_dim, 
                                                           self.num_out_hidden, 
                                                           ["w_0", "b_0"])
        
        self.weight, self.bias     = self._weight_and_bias(self.num_out_hidden, 
                                                           self.output_dim, 
                                                           ["w", "b"])
            
        # Flatten to apply same weights to all time steps.
        
        if self.model_type not in ['RETAIN', 'Seq2SeqAttention']: 
            
            rnn_output  = tf.reshape(rnn_output, [-1, self.num_out_hidden])
            
            prediction  = tf.nn.softmax(tf.matmul(rnn_output, self.weight) + self.bias)
        
        elif self.model_type == 'RETAIN':
            
            self.weight_a, self.bias_a = self._weight_and_bias(self.num_out_hidden, 1, ["w_a", "b_a"])
            
            rnn_output      = tf.reshape(rnn_output, [-1, self.num_out_hidden])
            
            self.attention  = tf.nn.softmax(tf.reshape(tf.matmul(rnn_output, self.weight_a) + self.bias_a, 
                                                       [-1, self.maximum_seq_length, 1]), axis=1)
            
            attn_mask       = tf.expand_dims(tf.sign(tf.reduce_max(tf.abs(self.rnn_input_), reduction_indices=2)), axis=2)
            masked_attn     = tf.multiply(attn_mask, self.attention)
            attn_norms      = tf.expand_dims(tf.tile(tf.reduce_sum(masked_attn, axis=1), [1, self.maximum_seq_length]), axis=2)
            self.attention  = masked_attn/attn_norms
            self.attention_ = tf.tile(self.attention, [1, 1, self.input_dim])
            self.context    = tf.reduce_sum(tf.multiply(self.attention_, self.rnn_input_), reduction_indices=1)
            context_layer   = tf.matmul(self.context, self.weight_0) + self.bias_0
            prediction      = tf.nn.softmax(tf.matmul(context_layer, self.weight) + self.bias)
        
        elif self.model_type == 'Seq2SeqAttention':
            
            self.attention  = self.seq2seq_attn
            
            attn_mask       = tf.expand_dims(tf.sign(tf.reduce_max(tf.abs(self.rnn_input_), reduction_indices=2)), axis=2)
            masked_attn     = tf.multiply(attn_mask, self.attention)
            attn_norms      = tf.expand_dims(tf.tile(tf.reduce_sum(masked_attn, axis=1), [1, self.maximum_seq_length]), axis=2)
            self.attention  = masked_attn/attn_norms
            self.attention_ = tf.tile(self.attention, [1, 1, self.input_dim])
            self.context    = tf.reduce_sum(tf.multiply(self.attention_, self.rnn_input_), reduction_indices=1)
            context_layer   = tf.matmul(self.context, self.weight_0) + self.bias_0
            prediction      = tf.nn.softmax(tf.matmul(context_layer, self.weight) + self.bias)

        prediction      = tf.reshape(prediction, [-1, self.maximum_seq_length, self.output_dim])
        self.predicted  = prediction
        self.predicted  = tf.identity(self.predicted, name=self.output_name)
        
        return prediction

    
    def process_rnn_inputs(self):
        
        if self.model_type in ['RETAIN', 'Seq2SeqAttention']: 
            
            self.num_samples = tf.shape(self.data)[0]
             
            Lengths_         = np.repeat(self.length, self.maximum_seq_length)
            
            conv_data        = tf.reshape(tf.tile(self.data, [1, self.maximum_seq_length, 1]), 
                                          [self.maximum_seq_length * self.num_samples, 
                                           self.maximum_seq_length, self.input_dim])
            
            conv_mask_       = tf.ones([self.maximum_seq_length, self.maximum_seq_length], tf.float32)
            
            conv_mask        = tf.tile(tf.expand_dims(tf.tile(tf.matrix_band_part(conv_mask_, -1, 0), 
                                                              [self.num_samples, 1]), 2), 
                                                              [1, 1, self.input_dim])
            
            masked_data   = tf.multiply(conv_data, conv_mask)
            
            Seq_lengths_  = tf.tile(tf.range(1, self.maximum_seq_length + 1, 1), [self.num_samples])
            
            if self.model_type == 'RETAIN':
            
                self.rnn_input_  = tf.reverse_sequence(masked_data, batch_axis=0, seq_dim=1, 
                                                       seq_lengths=Seq_lengths_, seq_axis=None)
            else:
                
                self.rnn_input_  = masked_data
                
            
            used         = tf.sign(tf.reduce_max(tf.abs(self.rnn_input_), reduction_indices=2))
            length       = tf.reduce_sum(used, reduction_indices=1)
            self.length_ = tf.cast(length, tf.int32)
            
            self.target_ = tf.tile(self.target, [self.maximum_seq_length, 1, 1]) 
            
        else:    
            
            self.rnn_input_ = self.rnn_input
            self.target_    = self.target 
            self.length_    = self.length

    @lazy_property
    def loss(self):
        
        # Compute cross entropy for each frame.
        cross_entropy  = tf.reduce_sum(-1*(self.target * tf.log(self.prediction) + (1-self.target)*(tf.log(1-self.prediction))),
                                       reduction_indices=2) 

        mask           = tf.sign(tf.reduce_max(tf.abs(self.data), reduction_indices=2))
        cross_entropy *= mask
        self.mask      = mask
        
        # Average over actual sequence lengths.
        cross_entropy  = tf.reduce_sum(cross_entropy, reduction_indices=1)
        cross_entropy /= tf.cast(self.length, tf.float32)
        
        return tf.reduce_mean(cross_entropy)
        
    
    
    def train(self, X, Y, T=None):
        
        X_, Y_   = padd_data(X, self.maximum_seq_length), padd_data(Y, self.maximum_seq_length)
        
        if T is not None:
            T_   = padd_data(T, self.maximum_seq_length)
        
        
        sess = tf.InteractiveSession()
        
        opt      = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.loss)
        init     = tf.global_variables_initializer()
        
        sess.run(init)

        saver = tf.train.Saver()
        
        for epoch in range(self.num_epochs):
                
            for _ in range(self.num_iterations):
                
                batch_samples = np.random.choice(list(range(X_.shape[0])), size=self.batch_size, replace=False)
                batch_train   = X_[batch_samples,:,:]
                batch_targets = Y_[batch_samples,:,:] 
                
                if T is not None:
                    batch_times = T_[batch_samples,:,:]
                    
                    train_dict  = {self.data   : batch_train,
                                   self.target : batch_targets,
                                   self.times  : batch_times}
                else:
                    
                    train_dict  = {self.data   : batch_train,
                                   self.target : batch_targets}
                
                
                sess.run(opt, feed_dict=train_dict)
                
                Loss          = sess.run(self.loss, feed_dict=train_dict)
                
                self._Losses.append(Loss)
 
                # Visualize function
                print('Epoch {} \t----- \tBatch {} \t----- \tLoss {}'.format(epoch, _, self._Losses[-1]))
  
        # change names
        saver.save(sess, "./mlaimRNN_model") 
        
        if os.path.exists("modelgraph"):
            
            shutil.rmtree("modelgraph")
        
        tf.saved_model.simple_save(sess, export_dir='modelgraph', inputs={"myInput": self.data}, 
                                   outputs={"myOutput": self.predicted})    
        
            
    def predict(self, X, T=None):
        
        with tf.Session() as sess:
            
            saver           = tf.train.import_meta_graph("mlaimRNN_model.meta")
            saver.restore(sess, tf.train.latest_checkpoint('./'))
            
            preds_lengths   = [len(X[k]) for k in range(len(X))]
            
            X_pred          = padd_data(X, padd_length=self.maximum_seq_length)
            
            if T is not None:
                T_pred      = padd_data_enforce(T, padd_length=self.maximum_seq_length)
                pred_dict   = {self.data   : X_pred, self.times   : T_pred}
            else:
                pred_dict   = {self.data   : X_pred}
            
            prediction_     = sess.run(self.prediction, pred_dict).reshape([-1, self.maximum_seq_length, 1])         

            preds_          = []
            
            for k in range(len(X)):
                
                preds_.append(prediction_[k, 0 : preds_lengths[k]])
                
                
            if self.model_type in ['RETAIN', 'Seq2SeqAttention']: 
                
                attn_                  = sess.run(self.attention, pred_dict) 
                attn_per_patient       = [attn_[u * self.maximum_seq_length : u * self.maximum_seq_length + self.maximum_seq_length, :, :] for u in range(len(X))]
                attn_lists_per_patient = [[attn_per_patient[u][k, 0 : k + 1, :] for k in range(self.maximum_seq_length)] for u in range(len(X))]
                
                preds_                 = (preds_, attn_lists_per_patient)
            
        return preds_    
    
    
    def evaluate(self, preds, Y_test):
        
        flat_preds   = flatten_sequences_to_numpy(preds)
        flat_Y_test  = np.array(list(itertools.chain.from_iterable([Y_test[k].tolist() for k in range(len(Y_test))])))
        
        _performance = roc_auc_score(flat_Y_test, flat_preds)
        
        return _performance
    
    @staticmethod
    def _weight_and_bias(in_size, out_size, wnames):
    
        weight = tf.get_variable(wnames[0], shape=[in_size, out_size], initializer=tf.contrib.layers.xavier_initializer())
        bias   = tf.get_variable(wnames[1], shape=[out_size], initializer=tf.contrib.layers.xavier_initializer())
        
        return weight, bias
        


class sequence_prediction:
    
    '''
    Parent class for all RNN models.
    '''
    
    def __init__(self, 
                 maximum_seq_length, 
                 input_dim, 
                 model_type='RNN',
                 rnn_type='RNN',
                 input_name="Input", 
                 output_name="Output",
                 model_name="SeqModel",
                 num_iterations=20, 
                 num_epochs=10, 
                 batch_size=100, 
                 learning_rate=0.0005, 
                 num_rnn_hidden=200, 
                 num_rnn_layers=1,
                 dropout_keep_prob=None,
                 num_out_hidden=200, 
                 num_out_layers=1,
                 verbosity=True,
                 **kwargs
                ):
        
        # Set all model variables

        self.maximum_seq_length = maximum_seq_length 
        self.input_dim          = input_dim
        self.output_dim         = input_dim
        self.model_type         = model_type
        self.rnn_type           = rnn_type
        
        self.input_name         = input_name 
        self.output_name        = output_name 
        self.model_name         = model_name
        self.num_iterations     = num_iterations
        self.num_epochs         = num_epochs
        self.batch_size         = batch_size
        self.learning_rate      = learning_rate
        self.num_rnn_hidden     = num_rnn_hidden
        self.num_rnn_layers     = num_rnn_layers
        self.dropout_keep_prob  = dropout_keep_prob
        self.num_out_hidden     = num_out_hidden
        self.num_out_layers     = num_out_layers
        self.verbosity          = verbosity
        
        
        self.build_rnn_model()
        tf.reset_default_graph()
        self.build_rnn_graph()
        
        
    
    def build_rnn_model(self):
        
        # replace this with dictionary style indexing
        
        model_options_names     = ['RNN','LSTM','GRU','PhasedLSTM']
        
        optimizer_options_names = []
        
        
        model_options   = [BasicRNNCell(self.num_rnn_hidden), rnn_cell.LSTMCell(self.num_rnn_hidden), 
                                   rnn_cell.GRUCell(self.num_rnn_hidden), PhasedLSTMCell(self.num_rnn_hidden)]
        
        self._rnn_model = model_options[np.where(np.array(model_options_names)==self.rnn_type)[0][0]]
        
        if self.dropout_keep_prob is not None:
            
            self._rnn_model = tf.nn.rnn_cell.DropoutWrapper(self._rnn_model, output_keep_prob=self.dropout_keep_prob)
        
        self._Losses = []
        

    def build_rnn_graph(self):
        
        self.data   = tf.placeholder(tf.float32, 
                                     [None, self.maximum_seq_length, self.input_dim], 
                                     name=self.input_name)
            
        self.target = tf.placeholder(tf.float32, 
                                     [None, self.maximum_seq_length, self.output_dim]) 
        
            
        self.rnn_input  = self.data 

            
    @lazy_property
    def length(self):
        
        used   = tf.sign(tf.reduce_max(tf.abs(self.data), reduction_indices=2))
        length = tf.reduce_sum(used, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        
        return length

    @lazy_property
    def prediction(self):
        
        self.process_rnn_inputs()
        
        # Recurrent network.   
        if self.model_type != 'Seq2SeqAttention': 
            
            rnn_output, _  = rnn.dynamic_rnn(self._rnn_model, 
                                             self.rnn_input_, 
                                             dtype=tf.float32, 
                                             sequence_length=self.length_,)
        else:
            
            try:
                
                tf.nn.seq2seq = tf.contrib.legacy_seq2seq
                tf.nn.rnn_cell = tf.contrib.rnn
                tf.nn.rnn_cell.GRUCell = tf.contrib.rnn.GRUCell
                print("TensorFlow version : >= 1.0")
            
            except: 
            
                print("TensorFlow version : 0.12")
            
            self.enc_inp    = [self.rnn_input_[:, t, :] for t in range(self.maximum_seq_length)]

            self.dec_output = [tf.placeholder(tf.float32, shape=(None, 1), 
                                              name="dec_output_".format(t)) for t in range(self.maximum_seq_length)]

            self.dec_inp    = [tf.zeros_like(self.enc_inp[0], dtype=np.float32, name="GO")] + self.enc_inp[:-1] 

            self.cells = []
    
            for i in range(self.num_rnn_layers):
                
                with tf.variable_scope('RNN_{}'.format(i)):
                    
                    self.cells.append(tf.nn.rnn_cell.GRUCell(self.num_rnn_hidden))
            
            
            self.cell  = tf.nn.rnn_cell.MultiRNNCell(self.cells)
            self.dec_outputs, self.dec_memory = tf.nn.seq2seq.basic_rnn_seq2seq(self.enc_inp, self.dec_inp, self.cell) 
            
            self.weight_dec, self.bias_dec = self._weight_and_bias(self.num_rnn_hidden, self.output_dim, ["w_dec", "b_dec"])
            
            self.seq2seq_attn = [(tf.matmul(i, self.weight_dec) + self.bias_dec) for i in self.dec_outputs]
            self.seq2seq_attn = tf.nn.softmax(tf.reshape(tf.stack(self.seq2seq_attn), 
                                                         [-1, self.maximum_seq_length, 1]), axis=1)
            
        
        # Softmax layer.
        self.weight_0, self.bias_0 = self._weight_and_bias(self.input_dim, 
                                                           self.num_out_hidden, 
                                                           ["w_0", "b_0"])
        
        self.weight, self.bias     = self._weight_and_bias(self.num_out_hidden, 
                                                           self.output_dim, 
                                                           ["w", "b"])
            
        # Flatten to apply same weights to all time steps.
        
        if self.model_type not in ['RETAIN', 'Seq2SeqAttention']: 
            
            rnn_output  = tf.reshape(rnn_output, [-1, self.num_out_hidden])
            
            prediction  = tf.matmul(rnn_output, self.weight) + self.bias
        
        elif self.model_type == 'RETAIN':
            
            self.weight_a, self.bias_a = self._weight_and_bias(self.num_out_hidden, 1, ["w_a", "b_a"])
            
            rnn_output      = tf.reshape(rnn_output, [-1, self.num_out_hidden])
            
            self.attention  = tf.nn.softmax(tf.reshape(tf.matmul(rnn_output, self.weight_a) + self.bias_a, 
                                                       [-1, self.maximum_seq_length, 1]), axis=1)
            
            attn_mask       = tf.expand_dims(tf.sign(tf.reduce_max(tf.abs(self.rnn_input_), reduction_indices=2)), axis=2)
            masked_attn     = tf.multiply(attn_mask, self.attention)
            attn_norms      = tf.expand_dims(tf.tile(tf.reduce_sum(masked_attn, axis=1), [1, self.maximum_seq_length]), axis=2)
            self.attention  = masked_attn #/attn_norms
            self.attention_ = tf.tile(self.attention, [1, 1, self.input_dim])
            self.context    = tf.reduce_sum(tf.multiply(self.attention_, self.rnn_input_), reduction_indices=1)
            context_layer   = tf.matmul(self.context, self.weight_0) + self.bias_0
            prediction      = tf.matmul(context_layer, self.weight) + self.bias
        
        elif self.model_type == 'Seq2SeqAttention':
            
            self.attention  = self.seq2seq_attn
            
            attn_mask       = tf.expand_dims(tf.sign(tf.reduce_max(tf.abs(self.rnn_input_), reduction_indices=2)), axis=2)
            masked_attn     = tf.multiply(attn_mask, self.attention)
            attn_norms      = tf.expand_dims(tf.tile(tf.reduce_sum(masked_attn, axis=1), [1, self.maximum_seq_length]), axis=2)
            self.attention  = masked_attn/attn_norms
            self.attention_ = tf.tile(self.attention, [1, 1, self.input_dim])
            self.context    = tf.reduce_sum(tf.multiply(self.attention_, self.rnn_input_), reduction_indices=1)
            context_layer   = tf.matmul(self.context, self.weight_0) + self.bias_0
            prediction      = tf.matmul(context_layer, self.weight) + self.bias

        prediction      = tf.reshape(prediction, [-1, self.maximum_seq_length, self.output_dim])
        self.predicted  = prediction
        self.predicted  = tf.identity(self.predicted, name=self.output_name)
        

        return prediction

    
    def process_rnn_inputs(self):
        
        if self.model_type in ['RETAIN', 'Seq2SeqAttention']: 
            
            self.num_samples = tf.shape(self.data)[0]
             
            Lengths_         = np.repeat(self.length, self.maximum_seq_length)
            
            conv_data        = tf.reshape(tf.tile(self.data, [1, self.maximum_seq_length, 1]), 
                                          [self.maximum_seq_length * self.num_samples, 
                                           self.maximum_seq_length, self.input_dim])
            
            conv_mask_       = tf.ones([self.maximum_seq_length, self.maximum_seq_length], tf.float32)
            
            conv_mask        = tf.tile(tf.expand_dims(tf.tile(tf.matrix_band_part(conv_mask_, -1, 0), 
                                                              [self.num_samples, 1]), 2), 
                                                              [1, 1, self.input_dim])
            
            masked_data   = tf.multiply(conv_data, conv_mask)
            
            Seq_lengths_  = tf.tile(tf.range(1, self.maximum_seq_length + 1, 1), [self.num_samples])
            
            if self.model_type == 'RETAIN':
            
                self.rnn_input_  = tf.reverse_sequence(masked_data, batch_axis=0, seq_dim=1, 
                                                       seq_lengths=Seq_lengths_, seq_axis=None)
            else:
                
                self.rnn_input_  = masked_data
                
            
            used         = tf.sign(tf.reduce_max(tf.abs(self.rnn_input_), reduction_indices=2))
            length       = tf.reduce_sum(used, reduction_indices=1)
            self.length_ = tf.cast(length, tf.int32)
            
            self.target_ = tf.tile(self.target, [self.maximum_seq_length, 1, 1]) 
            
        else:    
            
            self.rnn_input_ = self.rnn_input
            self.target_    = self.target 
            self.length_    = self.length

    @lazy_property
    def loss(self):

        MSE            = (self.target - self.prediction)**2
        
        return tf.reduce_mean(MSE) 
        
    
    def fit(self, X):
        
        self.normalizer   = MinMaxScaler()
        self.normalizer.fit(np.concatenate(X))

        self.X_normalized  = []
        self.Y_normalized  = []

        for k in range(len(X)):
            
            self.X_normalized.append(self.normalizer.transform(X[k])[:len(X[k])-1,:]) 
            self.Y_normalized.append(self.normalizer.transform(X[k])[1:,:]) 
        
        
        X_  = padd_data(self.X_normalized, self.maximum_seq_length)
        Y_  = padd_data(self.Y_normalized, self.maximum_seq_length)
        
        sess = tf.InteractiveSession()
        
        opt      = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.loss)
        init     = tf.global_variables_initializer()
        
        sess.run(init)

        saver = tf.train.Saver()
        
        for epoch in range(self.num_epochs):
                
            for _ in range(self.num_iterations):
                
                batch_samples = np.random.choice(list(range(X_.shape[0])), size=self.batch_size, replace=False)
                batch_train   = X_[batch_samples,:,:]
                batch_targets = Y_[batch_samples,:,:]
                    
                train_dict    = {self.data   : batch_train,
                                 self.target : batch_targets}
                
                
                sess.run(opt, feed_dict=train_dict)
                
                Loss          = sess.run(self.loss, feed_dict=train_dict)
                
                self._Losses.append(Loss)
 
                # Visualize function
                if self.verbosity:
                    print('Epoch {} \t----- \tBatch {} \t----- \tLoss {}'.format(epoch, _, self._Losses[-1]))
  
        # change names
        saver.save(sess, "./mlaimRNN_model") 
        
        if os.path.exists("modelgraph"):
            
            shutil.rmtree("modelgraph")
        
        tf.saved_model.simple_save(sess, export_dir='modelgraph', inputs={"myInput": self.data}, 
                                   outputs={"myOutput": self.predicted})    
        
            
    def predict(self, X, T=None):
        
        with tf.Session() as sess:
            
            saver           = tf.train.import_meta_graph("mlaimRNN_model.meta")
            saver.restore(sess, tf.train.latest_checkpoint('./'))
            
            preds_lengths   = [len(X[k]) for k in range(len(X))]

            self.X_normalized  = []


            for k in range(len(X)):
            
                self.X_normalized.append(self.normalizer.transform(X[k])[:len(X[k])-1,:]) 

            
            X_pred          = padd_data(self.X_normalized, padd_length=self.maximum_seq_length)

            pred_dict       = {self.data   : X_pred}
            
            prediction_     = sess.run(self.prediction, pred_dict).reshape([-1, self.maximum_seq_length, self.output_dim])         

            preds_          = []
            
            for k in range(len(X)):
                
                preds_.append(prediction_[k, 0 : preds_lengths[k], :])
                
                
            if self.model_type in ['RETAIN', 'Seq2SeqAttention']: 
                
                attn_                  = sess.run(self.attention, pred_dict) 
                attn_per_patient       = [attn_[u * self.maximum_seq_length : u * self.maximum_seq_length + self.maximum_seq_length, :, :] for u in range(len(X))]
                attn_lists_per_patient = [[attn_per_patient[u][k, 0 : k + 1, :] for k in range(self.maximum_seq_length)] for u in range(len(X))]
                
                preds_                 = (preds_, attn_lists_per_patient)
            
        return preds_    
    
    
    @staticmethod
    def _weight_and_bias(in_size, out_size, wnames):
    
        weight = tf.get_variable(wnames[0], shape=[in_size, out_size], initializer=tf.contrib.layers.xavier_initializer())
        bias   = tf.get_variable(wnames[1], shape=[out_size], initializer=tf.contrib.layers.xavier_initializer())
        
        return weight, bias

# BASE DATA LOADING

### Features currently included:
STATIC (from owner dataset):\
‘dog_id’ - for joining with health conditions dataset

CATEGORIES:
1. GENERAL HEALTH
2. EXCITABILITY
3. AGGRESSION
4. FEAR AND ANXIETY
5. SEPARATION-RELATED BEHAVIOR
6. ATTACHMENT, ATTENTION SEEKING
7. TRAINING DIFFICULTY
8. MISCELLANEOUS BEHAVIORS
9. DOGGO DEMOGRAPHICS-BREED
10. OTHER DEMOGRAPHICS
11. ENVIRONMENTAL
12. LIFESTYLE
13. MEDICATIONS / PREVENTIVES
14. TEMPORAL

GENERAL HEALTH\
[‘hs_general_health','Hs_recent_hospitalization']
- ‘hs_general_health' - 1 (Excellent) - 6 (Very poor)
- 'hs_recent_hospitalization' - true/false

EXCITABILITY\
['db_excitement_level_before_car_ride','db_excitement_level_before_walk']
- 'db_excitement_level_before_car_ride',  0.0 (calm) - 4.0 (extremely excitable)
- 'db_excitement_level_before_walk', 0.0 (calm) - 4.0 (extremely excitable)

AGGRESSION\
['db_aggression_level_approached_while_eating','db_aggression_level_delivery_workers_at_home','db_aggression_level_food_taken_away','db_aggression_level_on_leash_unknown_dog','db_aggression_level_on_leash_unknown_human', 'db_aggression_level_toys_taken_away','db_aggression_level_unknown_aggressive_dog', 'db_aggression_level_unknown_human_near_yard']
- 'db_aggression_level_approached_while_eating', 0.0 (none) - 4.0 (serious)
- 'db_aggression_level_delivery_workers_at_home', 0.0 (none) - 4.0 (serious)
- 'db_aggression_level_food_taken_away',  0.0 (none) - 4.0 (serious)
- 'db_aggression_level_on_leash_unknown_dog',  0.0 (none) - 4.0 (serious)
- 'db_aggression_level_on_leash_unknown_human',  0.0 (none) - 4.0 (serious)
- 'db_aggression_level_toys_taken_away',  0.0 (none) - 4.0 (serious)
- 'db_aggression_level_unknown_aggressive_dog',  0.0 (none) - 4.0 (serious)
- 'db_aggression_level_unknown_human_near_yard',  0.0 (none) - 4.0 (serious)

FEAR AND ANXIETY\
['db_fear_level_bathed_at_home','db_fear_level_loud_noises','db_fear_level_nails_clipped_at_home','db_fear_level_unknown_aggressive_dog','db_fear_level_unknown_dogs','db_fear_level_unknown_human_away_from_home','db_fear_level_unknown_human_touch','db_fear_level_unknown_objects_outside','db_fear_level_unknown_situations']
- 'db_fear_level_bathed_at_home',  0.0 (none) - 4.0 (extreme fear)
- 'db_fear_level_loud_noises',  0.0 (none) - 4.0 (extreme fear)
- 'db_fear_level_nails_clipped_at_home',   0.0 (none) - 4.0 (extreme fear)
- 'db_fear_level_unknown_aggressive_dog',  0.0 (none) - 4.0 (extreme fear)
- 'db_fear_level_unknown_dogs',  0.0 (none) - 4.0 (extreme fear)
- 'db_fear_level_unknown_human_away_from_home',   0.0 (none) - 4.0 (extreme fear)
- 'db_fear_level_unknown_human_touch',   0.0 (none) - 4.0 (extreme fear)
- 'db_fear_level_unknown_objects_outside',  0.0 (none) - 4.0 (extreme fear)
- 'db_fear_level_unknown_situations',  0.0 (none) - 4.0 (extreme fear)

SEPARATION-RELATED BEHAVIOR\
['db_left_alone_barking_frequency','db_left_alone_restlessness_frequency','db_left_alone_scratching_frequency']
- 'db_left_alone_barking_frequency',  0.0 (never) - 4.0 (always)
- 'db_left_alone_restlessness_frequency',  0.0 (never) - 4.0 (always)
- 'db_left_alone_scratching_frequency',  0.0 (never) - 4.0 (always)

ATTACHMENT, ATTENTION SEEKING\
['db_attention_seeking_follows_humans_frequency','db_attention_seeking_sits_close_to_humans_frequency']
- 'db_attention_seeking_follows_humans_frequency', 0.0 (never) - 4.0 (always)
- 'db_attention_seeking_sits_close_to_humans_frequency',  0.0 (never) - 4.0 (always)

TRAINING DIFFICULTY\
['db_training_distraction_frequency','db_training_obeys_sit_command_frequency','db_training_obeys_stay_command_frequency']
- 'db_training_distraction_frequency',  0.0 (never) - 4.0 (always)
- 'db_training_obeys_sit_command_frequency',  0.0 (never) - 4.0 (always)
- 'db_training_obeys_stay_command_frequency',  0.0 (never) - 4.0 (always)

MISCELLANEOUS BEHAVIORS\
['db_barks_frequency','db_chases_birds_frequency','db_chases_squirrels_frequency','db_chases_tail_frequency','db_chews_inappropriate_objects_frequency','db_defecates_alone_frequency','db_energetic_frequency','db_escapes_home_or_property_frequency','db_hyperactive_frequency','db_playful_frequency','db_pulls_leash_frequency','db_urinates_alone_frequency','db_urinates_in_home_frequency']
- 'db_barks_frequency', 0.0 (never) - 4.0 (always)
- 'db_chases_birds_frequency', 0.0 (never) - 4.0 (always)
- 'db_chases_squirrels_frequency', 0.0 (never) - 4.0 (always)
- 'db_chases_tail_frequency', 0.0 (never) - 4.0 (always)
- 'db_chews_inappropriate_objects_frequency',  0.0 (never) - 4.0 (always)
- 'db_defecates_alone_frequency',  0.0 (never) - 4.0 (always)
- 'db_energetic_frequency',  0.0 (never) - 4.0 (always)
- 'db_escapes_home_or_property_frequency',  0.0 (never) - 4.0 (always)
- 'db_hyperactive_frequency',   0.0 (never) - 4.0 (always)
- 'db_playful_frequency',   0.0 (never) - 4.0 (always)
- 'db_pulls_leash_frequency',   0.0 (never) - 4.0 (always)
- 'db_urinates_alone_frequency',   0.0 (never) - 4.0 (always)
- 'db_urinates_in_home_frequency'  0.0 (never) - 4.0 (always)

DOGGO DEMOGRAPHICS\ 
BREED (# representing a breed)\
['dd_breed_pure_or_mixed', 'dd_combined_main_breed']
- 'dd_breed_pure_or_mixed', 1 (purebred) 2 (mixed)
- 'dd_combined_main_breed', categorical # for breed - combines these two:
- - 'dd_breed_pure', categorical for breed, only when dog is purebred
- - 'dd_breed_pure_non_akc', categorical for breed, only when dog is purebred and isn't in the breed_pure list
- - 'dd_breed_mixed_primary', categorical for breed, only when dog is mixed

OTHER DEMOGRAPHICS\
['dd_age_years','dd_spayed_or_neutered','dd_spay_or_neuter_age','dd_insurance','dd_sex','dd_weight_lbs','dd_weight_range']
- 'dd_age_years' - 0.000 - 30.000 - take out all dogs with zero, since still growing/puppies
- 'dd_spayed_or_neutered', 1 YES 2 NO, but look out for NaN
- 'dd_spay_or_neuter_age', 1 (<6 mo), 2 (6-12mo), 3 (12-18mo), 4 (18-24mo), 5 (2-4yr), 6(4-6yr), 7(6-8yr), 8 (8+yr), 99(unknown)
- 'dd_insurance' - true/false
- 'dd_sex', 1 (M) 2(F)
- 'dd_weight_lbs', 0.0 - 150.0 (cotinuous)
- 'dd_weight_range' YES, need to get values for range? (categorical)
1- 0‐10 lbs\
2 - 11‐20 lbs\
3 - 21‐30 lbs\
4 - 31‐40 lbs\
5 - 41‐50 lbs\
6 - 51‐60 lbs\
7 - 61‐70 lbs\
8 - 71‐80 lbs\
9 - 81‐90 lbs\
10 - 91‐100 lbs\
11 - More than 100 lbs

ENVIRONMENTAL\
['de_air_cleaner_present','de_air_freshener_present','de_asbestos_present','de_central_air_conditioning_present','de_central_heat_present','de_routine_toys','de_daytime_sleep_avg_hours','de_nighttime_sleep_avg_hours','de_eats_feces','de_eats_grass_frequency','de_interacts_with_neighborhood_animals''de_interacts_with_neighborhood_humans','de_licks_chews_or_plays_with_non_toys']
- 'de_air_cleaner_present', 'de_air_freshener_present', 'de_asbestos_present', 'de_central_air_conditioning_present', 'de_central_heat_present', ALL, 1 YES, 0 NO, 99 UNKNOWN
- 'de_routine_toys' - 1 YES, 0 NO
- 'de_daytime_sleep_avg_hours', 0-24, 99 means unknown
- 'de_nighttime_sleep_avg_hours' 0-24, 99 means unknown
- 'de_eats_feces', 1 YES 0 NO 99 UNKNOWN
- 'de_eats_grass_frequency', 1 (frequently), 2 (infrequently), 0 (never)
- 'de_interacts_with_neighborhood_animals', 1 YES 0 NO
- 'de_interacts_with_neighborhood_humans', 1 YES 0 NO
- 'de_licks_chews_or_plays_with_non_toys', 1 YES 0 NO

LIFESTYLE\
['df_appetite','df_appetite_change_last_year','df_ever_malnourished','df_ever_overweight','df_ever_underweight','df_feedings_per_day','free_fed','df_diet_consistency','df_primary_diet_component''df_treats_frequency','df_weight_change_last_year']
- 'df_appetite',  1 (poor) 2 (avg) 3 (voracious)
- 'df_appetite_change_last_year', 1 YES 0 NO 99 UNKNOWN
- 'df_ever_malnourished', 1 YES 0 NO 99 UNKNOWN
- 'df_ever_overweight', 1 YES 0 NO 99 UNKNOWN
- 'df_ever_underweight', 1 YES 0 NO 99 UNKNOWN
- 'df_feedings_per_day', 1 (ONCE) 2 (TWICE) 3 (3+) 4 (Free fed) -> breakout free fed as separate data point 'free_fed'
- 'df_diet_consistency', 1 (very consistent) 2 (somewhat consistent) 3 not at all consistent) 98 OTHER
- 'df_treats_frequency', 3 (3+/day) 2 (2X) 1 (1X) 4 (occasionally, not daily) 0 never
- 'df_weight_change_last_year' 1 YES 0 NO 3 (dog is puppy still growing) 99 UNKNOWN
- 'df_primary_diet_component':\
1 - Commercially prepared dry food (kibble)\
2 - Commercially prepared canned food\
3 - Commercially prepared freeze‐dried food\
4 - Commercially prepared refrigerated or frozen raw food\
5 - Commercially prepared semi‐dry or semi‐moist food\
6 - Home prepared cooked diet\
7 - Home prepared raw diet\
98 - Other

MEDICATIONS / PREVENTIVES\
['mp_dental_examination_frequency','mp_dental_brushing_frequency','mp_dental_treat_frequency','mp_dental_food_frequency','mp_dental_breath_freshener_frequency','mp_flea_and_tick_treatment','mp_heartworm_preventative']
- 'mp_dental_examination_frequency', 0 (never) 1 (occasionally) 2 (monthly) 3 (weekly) 4 (daily)
- 'mp_dental_brushing_frequency', 0 (never) 1 (occasionally) 2 (monthly) 3 (weekly) 4 (daily)
- 'mp_dental_treat_frequency', 0 (never) 1 (occasionally) 2 (monthly) 3 (weekly) 4 (daily)
- 'mp_dental_food_frequency', 0 (never) 1 (occasionally) 2 (monthly) 3 (weekly) 4 (daily)
- 'mp_dental_breath_freshener_frequency', 0 (never) 1 (occasi) 2 (monthly) 3 (weekly) 4 (daily)
- 'mp_flea_and_tick_treatment', t/f
- 'mp_heartworm_preventative', t/f

TEMPORAL: (comes from health_conditions dataset)\
[‘dog_id’,‘hs_diagnosis_month’,‘hs_diagnosis_year’,‘hs_follow_up_ongoing',‘hs_required_surgery_or_hospitalization’]\
The main temporal data point is the condition that is diagnosis, which is basically data leakage, because it accounts for the diseases that will be predicted in the analysis.\
only have these available:
- ‘Dog_id’ -> to join to owner table
- ‘hs_diagnosis_month’	-> for reference
- ‘hs_diagnosis_year’ -> for reference
- ‘hs_follow_up_ongoing -> t/f
- ‘hs_required_surgery_or_hospitalization’ -> 1 (surgery) 2 (hospital) 3 (both) 4 (none)


### Features to predict:
['hs_health_conditions_skin','hs_health_conditions_eye','hs_health_conditions_cancer']

- 'hs_health_conditions_skin', 1 YES, 2 NO 
- 'hs_health_conditions_eye',  1 YES, 2 NO
- 'hs_health_conditions_cancer' -  1 YES, 2 NO

In [ ]:
# ORIGINAL DATA LOADING:
def doggo_data_loading(seq_length=2, apply_seq_length=True, use_surg=True, data_cat=[1]):
    """
    seq_length - the min # of sequences a dog can have to be included in temporal data
    apply_seq_length - whether or not the seq_length is applied
    use_surg - determines if the features include surgical visits in features data (part of temporal)
    
    data_cat parameter: takes an array of numbers representing these categories below.  
    Each one that is included is added to the model at training as a set of features.
    Default is to process them all!
    
    DATA CATEGORIES:
    1. GENERAL HEALTH * (2)
        [‘hs_general_health','hs_recent_hospitalization'] 
    2. EXCITABILITY (1)
        ['db_excitement_level_before_walk','db_excitement_level_before_car_ride'] 
        REMOVED (1): ,
    3. AGGRESSION (2) 
        ['db_aggression_level_approached_while_eating','db_aggression_level_on_leash_unknown_dog']
        REMOVED (5): ,'db_aggression_level_delivery_workers_at_home','db_aggression_level_food_taken_away','db_aggression_level_on_leash_unknown_human', 'db_aggression_level_toys_taken_away','db_aggression_level_unknown_aggressive_dog', 'db_aggression_level_unknown_human_near_yard'
    4. FEAR AND ANXIETY (2)
        ['db_fear_level_loud_noises','db_fear_level_unknown_human_touch']
        REMOVED (7): ,'db_fear_level_unknown_aggressive_dog','db_fear_level_bathed_at_home', 'db_fear_level_nails_clipped_at_home','db_fear_level_unknown_dogs','db_fear_level_unknown_human_away_from_home','db_fear_level_unknown_objects_outside','db_fear_level_unknown_situations'
    5. SEPARATION-RELATED BEHAVIOR (2)
        ['db_left_alone_barking_frequency','db_left_alone_restlessness_frequency']
        REMOVED (1): ,'db_left_alone_scratching_frequency'
    6. ATTACHMENT, ATTENTION SEEKING (2)
        ['db_attention_seeking_follows_humans_frequency','db_attention_seeking_sits_close_to_humans_frequency']
    7. TRAINING DIFFICULTY (2)
        ['db_training_distraction_frequency','db_training_obeys_stay_command_frequency']
        REMOVED (1): ,'db_training_obeys_sit_command_frequency'
    8. MISCELLANEOUS BEHAVIORS (4)
        ['db_barks_frequency','db_escapes_home_or_property_frequency','db_playful_frequency','db_urinates_in_home_frequency']
        REMOVED (9) : ,'db_chases_birds_frequency','db_chases_squirrels_frequency','db_chases_tail_frequency','db_defecates_alone_frequency','db_urinates_alone_frequency','db_pulls_leash_frequency','db_hyperactive_frequency','db_energetic_frequency'
    9. DOGGO DEMOGRAPHICS-BREED * (2)
        ['dd_breed_pure_or_mixed', 'dd_combined_main_breed']
    10. OTHER DEMOGRAPHICS * (6)
        ['dd_age_years','dd_spayed_or_neutered','dd_spay_or_neuter_age','dd_insurance','dd_sex','dd_weight_range']
        REMOVED (1): ,'dd_weight_lbs'
    11. ENVIRONMENTAL (4)
        ['de_daytime_sleep_avg_hours','de_nighttime_sleep_avg_hours','de_eats_feces','de_eats_grass_frequency']
        REMOVED (9): 'de_central_air_conditioning_present','de_central_heat_present','de_air_cleaner_present','de_air_freshener_present','de_asbestos_present','de_routine_toys','de_interacts_with_neighborhood_animals','de_interacts_with_neighborhood_humans','de_licks_chews_or_plays_with_non_toys'
    12. LIFESTYLE * (7)
        ['df_appetite','df_appetite_change_last_year','df_ever_malnourished','df_ever_overweight','df_feedings_per_day','free_fed']
        REMOVED (4): ,'df_diet_consistency','df_primary_diet_component','df_treats_frequency','df_ever_underweight','df_weight_change_last_year'
    13. MEDICATIONS / PREVENTIVES * (3)
        ['mp_dental_brushing_frequency','mp_flea_and_tick_treatment','mp_heartworm_preventative']
        REMOVED (4): 'mp_dental_examination_frequency',,'mp_dental_treat_frequency','mp_dental_food_frequency','mp_dental_breath_freshener_frequency'

    TEMPORAL (automatic) (2 features)
        [‘dog_id’,‘hs_diagnosis_month’,‘hs_diagnosis_year’,‘hs_follow_up_ongoing',‘hs_required_surgery_or_hospitalization’] 
    
    47 total, get rid of 1!
    
    PREDICTION CATEGORIES:
    'skin' - 'hs_health_conditions_skin', 1 YES, 2 NO
    'eye' - 'hs_health_conditions_eye', 1 YES, 2 NO
    'cancer' - 'hs_health_conditions_cancer' - 1 YES, 2 NO
    """
    
    # Data prep
    # pull up data
    heal_df = pd.read_csv("CSV_DATA/DAP_2020_HLES_health_condition_v1.1.csv")
    ownr_df = pd.read_csv("CSV_DATA/DAP_2020_HLES_dog_owner_v1.1.csv")
    
    # pull together data based on data_cat array:
    # also massage any data that needs massaging in each of these categories (only do if needed)
    data_list = []
    
    # add prediction columns:
    data_list.extend(('dog_id','hs_health_conditions_skin','hs_health_conditions_eye','hs_health_conditions_cancer'))
    
    #3
    
    temporal = []
    
    # ALT APPROACH:
    # data_list.extend(('hs_general_health',
    #                   'db_aggression_level_approached_while_eating',
    #                   'db_left_alone_restlessness_frequency',
    #                   'db_attention_seeking_follows_humans_frequency',
    #                   'db_training_obeys_stay_command_frequency',
    #                   'db_barks_frequency',
    #                   'db_escapes_home_or_property_frequency',
    #                   'db_urinates_in_home_frequency',
    #                   'dd_breed_pure_or_mixed',
    #                   'dd_breed_pure','dd_breed_pure_non_akc','dd_breed_mixed_primary',
    #                   'dd_age_years',
    #                   'dd_spayed_or_neutered',
    #                   'dd_spay_or_neuter_age',
    #                   'dd_insurance',
    #                   'dd_sex',
    #                   'dd_weight_range',
    #                   'de_eats_feces',
    #                   'df_appetite',
    #                   'mp_dental_brushing_frequency',
    #                   'mp_flea_and_tick_treatment',
    #                   'mp_heartworm_preventative'
    #                  ))
    # ALL
    data_list.extend(('dd_breed_pure_or_mixed',
                      'dd_breed_pure','dd_breed_pure_non_akc','dd_breed_mixed_primary','df_ever_underweight',
                    'hs_recent_hospitalization','dd_spayed_or_neutered','dd_spay_or_neuter_age',
                      'dd_insurance','de_daytime_sleep_avg_hours','de_nighttime_sleep_avg_hours','de_eats_feces',
                      'df_appetite','df_appetite_change_last_year','df_ever_malnourished','df_ever_overweight','df_feedings_per_day','free_fed',
                      'mp_flea_and_tick_treatment','mp_heartworm_preventative',
                      'df_primary_diet_component',
                      'mp_dental_brushing_frequency',
                      'db_excitement_level_before_walk','db_excitement_level_before_car_ride',
                      'db_aggression_level_approached_while_eating','db_aggression_level_on_leash_unknown_dog',
                      'db_left_alone_barking_frequency','db_left_alone_restlessness_frequency',
                      'db_fear_level_loud_noises','db_fear_level_unknown_human_touch',
                      'db_attention_seeking_follows_humans_frequency','db_attention_seeking_sits_close_to_humans_frequency',
                      'hs_general_health','dd_age_years','dd_sex','dd_weight_range',
                      'de_eats_grass_frequency',
                      'db_training_distraction_frequency','db_training_obeys_stay_command_frequency',
                      'db_barks_frequency','db_escapes_home_or_property_frequency','db_playful_frequency','db_urinates_in_home_frequency',
                     ))
    
    # REMOVE: 'hs_recent_hospitalization'
    # REMOVE: 'db_excitement_level_before_walk','db_excitement_level_before_car_ride'
    # REMOVE: 'db_aggression_level_on_leash_unknown_dog'
    # REMOVE: 'db_left_alone_barking_frequency',
    # REMOVE: 'db_fear_level_loud_noises','db_fear_level_unknown_human_touch'
    # REMOVE: 'db_attention_seeking_sits_close_to_humans_frequency'
    # REMOVE: 'db_training_distraction_frequency'
    # REMOVE: 'db_playful_frequency'
    # REMOVE: 'de_daytime_sleep_avg_hours','de_nighttime_sleep_avg_hours','de_eats_grass_frequency'
    # REMOVE: 'df_appetite_change_last_year','df_ever_malnourished','df_ever_overweight','df_feedings_per_day','free_fed'
    
#     if 1 in data_cat:
#         print('using GENERAL HEALTH') #2
#         data_list.extend(('hs_general_health','hs_recent_hospitalization'))
#     if 2 in data_cat:
#         print('using EXCITABILITY') #2
#         data_list.extend(('db_excitement_level_before_walk','db_excitement_level_before_car_ride'))
#     if 3 in data_cat:
#         print('using AGGRESSION') #2
#         data_list.extend(('db_aggression_level_approached_while_eating','db_aggression_level_on_leash_unknown_dog'))
#     if 4 in data_cat:
#         print('using FEAR AND ANXIETY') #2
#         data_list.extend(('db_fear_level_loud_noises','db_fear_level_unknown_human_touch'))
#     if 5 in data_cat:
#         print('using SEPARATION-RELATED BEHAVIOR') #2
#         data_list.extend(('db_left_alone_barking_frequency','db_left_alone_restlessness_frequency'))
#     if 6 in data_cat:
#         print('using ATTACHMENT, ATTENTION SEEKING') #2
#         data_list.extend(('db_attention_seeking_follows_humans_frequency','db_attention_seeking_sits_close_to_humans_frequency'))
#     if 7 in data_cat:
#         print('using TRAINING DIFFICULTY') #2
#         data_list.extend(('db_training_distraction_frequency','db_training_obeys_stay_command_frequency'))
#     if 8 in data_cat:
#         print('using MISCELLANEOUS BEHAVIORS') #4
#         data_list.extend(('db_barks_frequency','db_escapes_home_or_property_frequency','db_playful_frequency','db_urinates_in_home_frequency'))
#     if 9 in data_cat:
#         print('using DEMOGRAPHICS-BREED') #2 (consolidated from 4)
#         data_list.extend(('dd_breed_pure_or_mixed','dd_breed_pure','dd_breed_pure_non_akc','dd_breed_mixed_primary'))        
#     if 10 in data_cat:
#         print('using OTHER DEMOGRAPHICS') #6
#         data_list.extend(('dd_age_years','dd_spayed_or_neutered','dd_spay_or_neuter_age','dd_insurance','dd_sex','dd_weight_range'))
#     if 11 in data_cat:
#         print('using ENVIRONMENTAL')  #4
#         data_list.extend(('de_daytime_sleep_avg_hours','de_nighttime_sleep_avg_hours','de_eats_feces','de_eats_grass_frequency'))
#     if 12 in data_cat:
#         print('using LIFESTYLE') #6
#         data_list.extend(('df_appetite','df_appetite_change_last_year','df_ever_malnourished','df_ever_overweight','df_feedings_per_day','free_fed'))
#     if 13 in data_cat: 
#         print('using MEDICATIONS / PREVENTIVES') #3
#         data_list.extend(('mp_dental_brushing_frequency','mp_flea_and_tick_treatment','mp_heartworm_preventative'))
    
    # make temporal automatic
    print('using TEMPORAL')
    temporal.extend(('dog_id','hs_condition_type','hs_follow_up_ongoing','hs_required_surgery_or_hospitalization')) # 'hs_diagnosis_month','hs_diagnosis_year',
    
    feat_num_static = len(data_list)
    # these are not features (so reduce by 2):
    # 'dog_id', 'hs_health_conditions_skin','hs_health_conditions_eye','hs_health_conditions_cancer'
    # will use the two health conditions that are not being predicted as features!
    feat_num_static = feat_num_static - 2
    feat_list = data_list
    # print('feat_num_static: ', feat_num_static)
#     print('static featurse: ', data_list)
    
    feat_num_temporal = len(temporal)
    # these are not features (so reduce by 1): 'dog_id'
    feat_num_temporal = feat_num_temporal - 1
    feat_list.extend(temporal)
    # print('feat_num_temporal: ', feat_num_temporal)
#     print('temporal features: ', temporal)
    feat_num = feat_num_static + feat_num_temporal
    # print('feat_num total initial: ', feat_num)
#     print('feat_list:', feat_list)
    
    selected_df = ownr_df[ownr_df.columns.intersection(data_list)]
    temporal_df = heal_df[heal_df.columns.intersection(temporal)]
    
    # join dataframes using doggo id
    joined_df = selected_df.set_index('dog_id').join(temporal_df.set_index('dog_id'))
    # NOTE! Might want to take out these: 'hs_follow_up_ongoing','hs_required_surgery_or_hospitalization'
    #    BC those are usually a result of the disease and so a form of data leakage
    
    # set up temporal based disease stage
    joined_df['skin_dev'] = (joined_df['hs_condition_type']==4).astype(int)
    joined_df['eye_dev'] = (joined_df['hs_condition_type']==1).astype(int)
    
    # pull from cancer db and cross reference with dog id
    # Can get month and year from: 'hs_follow_up_ongoing', 'hs_initial_diagnosis_month', 'hs_initial_diagnosis_year'
    # tie those to the month and year in the health sequence data.
    
    # joined_df['cancer_dev'] = (joined_df['hs_condition_type']==4).astype(int)
    
    # print('dataframe shape 1: ', joined_df.shape)# [0]
    # print('feature data points: ', feat_num)
    
    # Update dog breeds if that category selected (have to before dropping NaN)
    if 9 in data_cat:
        # print('removing breed cols')
        joined_df['dd_breed_pure'] = joined_df['dd_breed_pure'].fillna(0)
        joined_df['dd_breed_pure_non_akc'] = joined_df['dd_breed_pure_non_akc'].fillna(0)
        joined_df['dd_breed_mixed_primary'] = joined_df['dd_breed_mixed_primary'].fillna(0)
        
        # add together for new
        temp_df = joined_df[['dd_breed_pure','dd_breed_pure_non_akc','dd_breed_mixed_primary']]
        sum_df = temp_df.sum(axis=1)
        joined_df['dd_combined_main_breed'] = sum_df
        feat_list.append('dd_combined_main_breed')
        
        # remove old columns
        drop_cols = ['dd_breed_pure','dd_breed_pure_non_akc','dd_breed_mixed_primary']
        joined_df = joined_df.drop(columns=drop_cols) # , axis=1
        
        # remove these, update feat_num (only by 2 though, since adding a new one)
        feat_num = feat_num - 2
        feat_list.remove('dd_breed_pure')
        feat_list.remove('dd_breed_pure_non_akc')
        feat_list.remove('dd_breed_mixed_primary')
    
    # clean out NaN
    # for temporal data, if all NaN, then just means wasn't in health records, will take out for now,
    #    but may need to come back and include as more samples of healthy dogs
    joined_df = joined_df.dropna()
    
    # reduce to only dogs with seq_lenth+ of records # turning off for now
#     joined_df['dog_id'] = joined_df.index  
    joined_df = joined_df.reset_index(level=0)
    if apply_seq_length==True:
        joined_df = joined_df[joined_df.groupby('dog_id')['dog_id'].transform('size') >= seq_length]
    else: 
        joined_df = joined_df
        
     # massage data into format better for analysis
    if 1 in data_cat:
        # GENERAL HEALTH'
        # 'hs_general_health','hs_recent_hospitalization'
        # convert t/f to 1/0
        joined_df['hs_recent_hospitalization'] = joined_df['hs_recent_hospitalization'].astype(int)    
    
    if 10 in data_cat:
        # OTHER DEMOGRAPHICS
        # 'dd_age_years','dd_spayed_or_neutered','dd_spay_or_neuter_age','dd_insurance',
        # 'dd_sex','dd_weight_lbs','dd_weight_range'
        joined_df['dd_insurance'] = joined_df['hs_follow_up_ongoing'].astype(int)
        joined_df['dd_spayed_or_neutered'] = joined_df['dd_spayed_or_neutered'].astype(int)
        # TODO - add these back in some capacity!
        # for now, remove unknown neutered age: dd_spay_or_neuter_age
        joined_df.drop(joined_df[joined_df.dd_spay_or_neuter_age == 99.0].index, inplace=True)
        
    if 11 in data_cat:
        # ENVIRONMENTAL 
        # ['de_daytime_sleep_avg_hours','de_nighttime_sleep_avg_hours','de_eats_feces','de_eats_grass_frequency']
        # 'de_air_cleaner_present','de_air_freshener_present','de_asbestos_present',
        # 'de_central_air_conditioning_present','de_central_heat_present','de_routine_toys',
        # 'de_daytime_sleep_avg_hours','de_nighttime_sleep_avg_hours',
        # 'de_eats_feces','de_eats_grass_frequency','de_interacts_with_neighborhood_animals',
        # 'de_interacts_with_neighborhood_humans','de_licks_chews_or_plays_with_non_toys'
        # joined_df['de_air_cleaner_present'] = np.where(joined_df['de_air_cleaner_present'] == 99, 0, 1)
        # joined_df['de_air_freshener_present'] = np.where(joined_df['de_air_freshener_present'] == 99, 0, 1)
        # joined_df['de_asbestos_present'] = np.where(joined_df['de_asbestos_present'] == 99, 0, 1)
        # joined_df['de_central_air_conditioning_present'] = np.where(joined_df['de_central_air_conditioning_present'] == 99, 0, 1)
        # joined_df['de_central_heat_present'] = np.where(joined_df['de_central_heat_present'] == 99, 0, 1)        
        # joined_df['de_routine_toys'] = joined_df['de_routine_toys'].astype(int)
        joined_df['de_daytime_sleep_avg_hours'] = np.where(joined_df['de_daytime_sleep_avg_hours'] == 99, 0, 1)  
        joined_df['de_nighttime_sleep_avg_hours'] = np.where(joined_df['de_nighttime_sleep_avg_hours'] == 99, 0, 1)  
        joined_df['de_eats_feces'] = np.where(joined_df['de_eats_feces'] == 99, 0, 1)  
#         'de_eats_grass_frequency', 1 (frequently), 2 (infrequently), 0 (never)        
        # joined_df['de_interacts_with_neighborhood_animals'] = joined_df['de_interacts_with_neighborhood_animals'].astype(int)
        # joined_df['de_interacts_with_neighborhood_humans'] = joined_df['de_interacts_with_neighborhood_humans'].astype(int)
        # joined_df['de_licks_chews_or_plays_with_non_toys'] = joined_df['de_licks_chews_or_plays_with_non_toys'].astype(int)

    if 12 in data_cat:
        # LIFESTYLE
        # ['df_appetite','df_appetite_change_last_year','df_ever_malnourished','df_ever_overweight',
        #  'df_ever_underweight','df_feedings_per_day','free_fed','df_weight_change_last_year']
        # 'df_appetite','df_appetite_change_last_year','df_ever_malnourished','df_ever_overweight',
        # 'df_ever_underweight','df_feedings_per_day','free_fed','df_diet_consistency',
        # 'df_primary_diet_component','df_treats_frequency','df_weight_change_last_year'
        # if don't know one of these, going to assume a no
        joined_df['df_appetite_change_last_year'] = np.where(joined_df['df_appetite_change_last_year'] == 99, 0, 1)
        joined_df['df_ever_malnourished'] = np.where(joined_df['df_ever_malnourished'] == 99, 0, 1)
        joined_df['df_ever_overweight'] = np.where(joined_df['df_ever_overweight'] == 99, 0, 1)
        joined_df['df_ever_underweight'] = np.where(joined_df['df_ever_underweight'] == 99, 0, 1)
        joined_df['df_primary_diet_component'] = np.where(joined_df['df_primary_diet_component'] == 98, 8, 1)
        
        # 'df_feedings_per_day', 1 (ONCE) 2 (TWICE) 3 (3+) 4 (Free fed) -> breakout free fed as separate data point 'free_fed'
        # break out the free fed:
        # set up bool field for t/f 1/0 'free_fed'
        joined_df['free_fed'] = (joined_df['df_feedings_per_day']==4).astype(int)
        joined_df['df_feedings_per_day'] = np.where(joined_df['df_feedings_per_day'] == 4, 0, 1)
        # added a feature
        feat_num = feat_num + 1
        
    if 13 in data_cat:
        # MEDICATIONS / PREVENTIVES'
        # 'mp_dental_examination_frequency','mp_dental_brushing_frequency','mp_dental_treat_frequency',
        # 'mp_dental_food_frequency','mp_dental_breath_freshener_frequency','mp_flea_and_tick_treatment',
        # 'mp_heartworm_preventative'
        joined_df['mp_flea_and_tick_treatment'] = joined_df['mp_flea_and_tick_treatment'].astype(int)
        joined_df['mp_heartworm_preventative'] = joined_df['mp_heartworm_preventative'].astype(int)
#         'mp_flea_and_tick_treatment', t/f
#         'mp_heartworm_preventative', t/f
    
    # TEMPORAL
    # 'hs_follow_up_ongoing','hs_required_surgery_or_hospitalization'
    joined_df['hs_follow_up_ongoing'] = joined_df['hs_follow_up_ongoing'].astype(int)
    # want 'hs_required_surgery_or_hospitalization' to be more linear in nature (ie. 1 is nothing, 4 is worse (surgery AND hospitalization))
    for index, row in joined_df.iterrows(): 
        if row['hs_required_surgery_or_hospitalization'] == 1.0:
            row['state'] = 3
        elif row['hs_required_surgery_or_hospitalization'] == 3.0:
            row['state'] = 4
        elif row['hs_required_surgery_or_hospitalization'] == 4.0:
            row['state'] = 1
        else:
            row['state'] = 2
    
    # convert prediction columns to traditional bool:
    joined_df['hs_health_conditions_skin'] = np.where(joined_df['hs_health_conditions_skin'] == 2, 0, 1)
    joined_df['hs_health_conditions_eye'] = np.where(joined_df['hs_health_conditions_eye'] == 2, 0, 1)
    joined_df['hs_health_conditions_cancer'] = np.where(joined_df['hs_health_conditions_cancer'] == 2, 0, 1) 
    
    # take these out of features list, bc need to process separately or aren't features
#     feat_list.remove('dog_id')
    # Need logic to take these out when applicable: 'hs_health_conditions_eye', 'hs_health_conditions_cancer' (but leave in others for features)
    del feat_list[0:1]
    feat_list.remove('dog_id')
    # HAVE TO KEEP SKIN FOR TimeGANs!  But remove later where not wanted.
    
    final_df = joined_df
    
    
    
    
    
#     # reduce to just dogs with 4+ health records
#     # then look at the most common conditions & most common surgical/hospitalization scenarios
#     sm_hl_df = heal_df[['dog_id', 'hs_condition_type','hs_required_surgery_or_hospitalization']]

#     # MOST COMMON ISSUES:
#     filter_df = sm_hl_df.loc[(sm_hl_df['hs_condition_type']==4) | (sm_hl_df['hs_condition_type']==3) | (sm_hl_df['hs_condition_type']==18) | (sm_hl_df['hs_condition_type']==16) | (sm_hl_df['hs_condition_type']==11) | (sm_hl_df['hs_condition_type']==1),['dog_id', 'hs_condition_type','hs_required_surgery_or_hospitalization']]

#     # reduce to only dogs with seq_lenth+ of records # turning off for now
#     if apply_seq_length==True:
#         segmin_df = filter_df[filter_df.groupby('dog_id')['dog_id'].transform('size') > seq_length]
#     else: 
#         segmin_df = filter_df

#     # turn values into binary, adding new columns
#     # if has 4, should be skin 1
#     segmin_df['dis_skin'] = (segmin_df['hs_condition_type']==4).astype(int)
#     segmin_df['dis_oral'] = (segmin_df['hs_condition_type']==3).astype(int)
#     segmin_df['dis_trau'] = (segmin_df['hs_condition_type']==18).astype(int)
#     segmin_df['dis_infe'] = (segmin_df['hs_condition_type']==16).astype(int)
#     segmin_df['dis_bone'] = (segmin_df['hs_condition_type']==11).astype(int)
#     segmin_df['dis_eyes'] = (segmin_df['hs_condition_type']==1).astype(int)
    
#     final_df = segmin_df
#     if use_surg==True:
#         segmin_df['surgery'] = (segmin_df['hs_required_surgery_or_hospitalization']==1).astype(int)
#         segmin_df['hospitalized'] = (segmin_df['hs_required_surgery_or_hospitalization']==2).astype(int)
#         segmin_df['both_surg_hosp'] = (segmin_df['hs_required_surgery_or_hospitalization']==3).astype(int)
#         # remove old columns
#         final_df = segmin_df[['dog_id','dis_skin','dis_oral','dis_trau','dis_infe','dis_bone','dis_eyes','surgery','hospitalized','both_surg_hosp','hs_required_surgery_or_hospitalization']] 
#     else:
#         # remove old columns
#         final_df = segmin_df[['dog_id','dis_skin','dis_oral','dis_trau','dis_infe','dis_bone','dis_eyes','hs_required_surgery_or_hospitalization']] 

#     feat_num = 0
#     feat_list = []
    
    return final_df, feat_num, feat_list 


# Time-series Generative Adversarial Networks

In [ ]:
## Min Max Normalizer

def MinMaxScaler3(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

In [ ]:
def doggo_data_timeGANs(df):
    
    # remove non-feature columns
    drop_cols = ['dog_id']
    df = df.drop(drop_cols, axis=1)
    
    # turn into segment vector
    x = df.to_numpy()
    
    # Flip the data to make chronological data
    # this reverses the order of the entire dataset, so newest in sequence is now first
    # but data itself is still intact
    x = x[::-1]
    
    # Min-Max Normalizer
    x = MinMaxScaler3(x)
    
    # Build dataset
    dataX = []
    
    for i in range(0, len(x) - seq_length):
        _x = x[i:i + seq_length]
        dataX.append(_x)
        
    # Mix Data (to make it similar to random sample)
    idx = np.random.permutation(len(dataX))
    
    outputX = []
    for i in range(len(dataX)):
        outputX.append(dataX[idx[i]])
        
     # load final vars
    X_ = outputX 
    
    return outputX

In [ ]:
## Main Parameters
# Experiments iterations
Iteration = 2 # started with: 1
Sub_Iteration = 3 # started with: 2

## Data Loading
seq_length = 4
data_cats = [1,2,3,4,5,6,7,8,9,10,11,12,13]

gan_df, gan_feat_num, gan_feat_list = doggo_data_loading(seq_length=seq_length, data_cat=data_cats)
dataX = doggo_data_timeGANs(gan_df)
    
print('doggo dataset is ready with features: ', gan_feat_num)
# df.sample(28)
# df.head(28)
print(len(gan_feat_list))
print(gan_feat_list)
gan_df.head(10)

In [ ]:
## Newtork Parameters
parameters = dict()

parameters['hidden_dim'] = len(dataX[0][0,:]) * 4
parameters['num_layers'] = 3 # started with: 2 (has to be at least 2!)
parameters['iterations'] = 10000 # started with: 1000, low as 10
parameters['batch_size'] = 128 # started with: 64, low as 2
parameters['module_name'] = 'gru'   # Other options: 'lstm' or 'lstmLN'
parameters['z_dim'] = len(dataX[0][0,:]) 

print('Parameters are ' + str(parameters))

## Experiments
# Output Initialization
Discriminative_Score = list()
Predictive_Score = list()

In [ ]:
# Run TimeGANs model
print('Start iterations') 
    
# Each Iteration
for it in range(Iteration):

    
    # Synthetic Data Generation
    dataX_hat = timegan(dataX, parameters)   
      
    print('Finish Synthetic Data Generation')

    ## Performance Metrics
    
    # 1. Discriminative Score
    Acc = list()
    for tt in range(Sub_Iteration):
        Temp_Disc = discriminative_score_metrics (dataX, dataX_hat)
        Acc.append(Temp_Disc)
    
    Discriminative_Score.append(np.mean(Acc))
    
    # 2. Predictive Performance
    MAE_All = list()
    for tt in range(Sub_Iteration):
        MAE_All.append(predictive_score_metrics (dataX, dataX_hat))
        
    Predictive_Score.append(np.mean(MAE_All))    
    
print('Finish TimeGAN iterations')

In [ ]:
PCA_Analysis (dataX, dataX_hat)

In [ ]:
tSNE_Analysis (dataX, dataX_hat)

In [ ]:
# Print Results
print('Discriminative Score - Mean: ' + str(np.round(np.mean(Discriminative_Score),4)) + ', Std: ' + str(np.round(np.std(Discriminative_Score),4)))
print('Predictive Score - Mean: ' + str(np.round(np.mean(Predictive_Score),4)) + ', Std: ' + str(np.round(np.std(Predictive_Score),4)))

In [ ]:
# dataX[0:10]
# for i in dataX[0:20]:
#     print(i.shape)
    
# print(dataX[0].shape)
# dataX[0]

In [ ]:
# print(dataX[1].shape)
# dataX[1]

In [ ]:
print(dataX[2].shape)
dataX[2]

# DISEASE TRAJECTORY STATE PREDICTIONS

In [ ]:
def generate_doggo_trajectory(df, feats, predict='skin'):
    # print(assm_df.head(5))
    
    list_arr = []
    arry_list = np.array([])
    list_list = []
    list_feat = []
    cur_dog_id = df.iloc[0]['dog_id']
    
    df['state'] = 0.0
    state_list = []
    list_state_list = []
    
    for index, row in df.iterrows(): 
        # set up state lists - 'hs_health_conditions_?????'
        # start with just skin - 'hs_health_conditions_skin', 1 YES, 2 NO
        # set up state lists
        if row['hs_health_conditions_skin'] == 1.0:
            row['state'] = 1
        elif row['hs_health_conditions_skin']  == 2.0:
            row['state'] = 0
        
        
        
        if cur_dog_id != row['dog_id']:
            # print('list_list: ', list_list)
            arry_list = np.array(list_list)
            # print('arry_list: ', arry_list)
            list_arr.append(arry_list)
            list_state_list.append(state_list)
        
            # reset various fields
            state_list = []
            arry_list = np.array([]) # shouldn't need this, but jic
            list_list = [] # reset the list of lists
            cur_dog_id = row['dog_id']

        # set up features list:
        # hard code this for now, then go back and iterate it:
        # [‘hs_general_health','hs_recent_hospitalization']
        # [‘hs_follow_up_ongoing',‘hs_required_surgery_or_hospitalization’]
        # ['hs_health_conditions_skin','hs_health_conditions_eye','hs_health_conditions_cancer']
        
#         list_feat = [row['dis_skin'],row['dis_oral'], row['dis_trau'], row['dis_infe'], row['dis_bone'], row['dis_eyes']]
        # list_feat = [row['hs_general_health'],row['hs_recent_hospitalization'], row['hs_follow_up_ongoing'], row['hs_required_surgery_or_hospitalization'], row['hs_health_conditions_eye'], row['hs_health_conditions_cancer']]
        # print('pre feats list_feat', list_feat)
        # print(feats)
        for feat in feats:
            # print(feat)
            list_feat.append(row[feat])
        # print('list_feat: ', list_feat)
        # print('list_feat size: ', len(list_feat))
        list_list.append(list_feat)
        # print(list_list)
        state_list.append(row['state'])
        list_feat = [] # shouldn't need this, but jic
    
    # if predict=='skin':
    #     drop_cols = ['hs_health_conditions_skin']
    #     df.drop(drop_cols, axis=1)
    # elif predict=='eye':
    #     drop_cols = ['hs_health_conditions_eye']
    #     df.drop(drop_cols, axis=1)            
    # elif predict=='cancer':
    #     drop_cols = ['hs_health_conditions_cancer']
    #     df.drop(drop_cols, axis=1)
    
    # load final vars
    X_ = list_arr
    # arrX = np.array(X_)
    # print('x shape: ', arrX.shape)
    
    S_ = list_state_list
    # arrS = np.array(S_)
    # print('x shape: ', arrS.shape)
    
#     print(arrX[0:20])
#     print(arrS[0:20])
    
    return X_, S_ 

In [ ]:
    # drop_cols = ['dog_id', 'hs_health_conditions_skin']
    # df.drop(drop_cols, axis=1)

#     if predict=='skin':
#         joined_df['hs_health_conditions_skin'] = np.where(joined_df['hs_health_conditions_skin'] == 2, 0, 1)
#     elif predict=='eye':
#         joined_df['hs_health_conditions_eye'] = np.where(joined_df['hs_health_conditions_eye'] == 2, 0, 1)
#     elif predict=='cancer':
#         joined_df['hs_health_conditions_cancer'] = np.where(joined_df['hs_health_conditions_cancer'] == 2, 0, 1)

# if predict=='skin':
        #     row['state'] = row['hs_health_conditions_skin']
        #     list_feat = [row['hs_health_conditions_eye'],row['hs_health_conditions_cancer']]
        # elif predict=='eye':
        #     row['state'] = row['hs_health_conditions_eye']
        #     list_feat = [row['hs_health_conditions_skin'],row['hs_health_conditions_cancer']]           
        # elif predict=='cancer':
        #     row['state'] = row['hs_health_conditions_cancer']
        #     list_feat = [row['hs_health_conditions_eye'],row['hs_health_conditions_skin']]
        
        # print('pre feats list_feat', list_feat)
        # print(feats)
        # for feat in feats:
        #     # print(feat)
        #     list_feat.append(row[feat])
        # print(list_feat)        


In [ ]:
## Data Loading
seq_length = 4
data_cats = [1,2,3,4,5,6,7,8,9,10,11,12,13] # 
disease = 'skin' # 'eye' 'cancer'

# generate the data
# df, feat_num, feat_list = doggo_data_loading(seq_length=seq_length, data_cat=data_cats) # [1,2,3,4,5,6,7,8,9,10,11,12,13] # [1,10,14]
# final_df, feat_num, feat_list = doggo_data_loading(apply_seq_length=False, use_surg=False, data_cat=data_cats)
assm_df, assm_feat_num, assm_feat_list = doggo_data_loading(seq_length=seq_length, apply_seq_length=True, data_cat=data_cats) #  apply_seq_length=True, use_surg=False, 
print(assm_feat_num, ' : ', assm_feat_list)
# print(assm_df.shape)
# FYI - can't get entire DF to run through without ASSM collapsing.  Max on minimum params is 21,000, larger params 29,400
X_observations, true_states = generate_doggo_trajectory(assm_df[0:17000], feats=assm_feat_list, predict=disease)
# [0:17000]
# TODO adjust 0:20000 to as high as possible
# keep adding categories!

print(assm_df.columns)
print(len(assm_df.columns))
assm_df.head(10)
# X_observations[0:3]
# true_states[0:10]

# ## Data Loading
# seq_length = 4
# data_cats = [1]

# df, feat_num, feat_list = doggo_data_loading(seq_length=seq_length, data_cat=data_cats)

In [ ]:
trajectory_index = 3

true_states[trajectory_index]

In [ ]:
# Visualizing the hidden state data trajectories
fig, ax1 = plt.subplots()

t = list(range(len(true_states[trajectory_index])))

color = 'tab:red'
ax1.set_xlabel('Time step')
ax1.set_ylabel('Observations', color=color)
ax1.plot(t, X_observations[trajectory_index], color=color, linewidth=5, alpha=0.2)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  

color = 'tab:blue'
ax2.set_ylabel('Hidden states', color=color)  
ax2.step(t, true_states[trajectory_index], color=color, linewidth=5)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yticks([0, 1, 2])

fig.tight_layout()  
plt.show()

In [ ]:
print(assm_feat_num)
model = attentive_state_space_model(num_states=2,
                              maximum_seq_length=30, 
                              input_dim=assm_feat_num, # 6,  # started with: 6. tried 10, but broke, but that was original, adjust for how many features used in data
                              rnn_type='LSTM',
                              latent=True,
                              generative=True,
                              num_iterations=50,   # started with: 5
                              num_epochs=3,  # started with: 2
                              batch_size=100,  # started with: 10
                              learning_rate=5*1e-4, 
                              num_rnn_hidden=100,  # started with: 5
                              num_rnn_layers=1,
                              dropout_keep_prob=None,
                              num_out_hidden=100,  # started with: 5
                              num_out_layers=1)

In [ ]:
model.fit(X_observations)

In [ ]:
model.initial_probabilities

In [ ]:
model.transition_matrix

In [ ]:
model.state_means.T 

In [ ]:
model.state_covars

In [ ]:
X_observations[3]

In [ ]:
predict_idx = 0
state_inference, expected_observations, attention = model.predict([X_observations[predict_idx]])

In [ ]:
np.argmax(state_inference[predict_idx], axis=1)

In [ ]:
fig, ax1 = plt.subplots()

trajectory_index = 0

t = list(range(len(true_states[trajectory_index])))

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('MAP state estimate', color=color)
ax1.plot(t, np.argmax(state_inference[trajectory_index], axis=1), color=color, linewidth=5)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_yticks([0, 1, 2])

ax2 = ax1.twinx()  

color = 'tab:blue'
ax2.set_ylabel('True states', color=color)  
ax2.step(t, true_states[trajectory_index], color=color, linewidth=5)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yticks([0, 1, 2])

fig.tight_layout()  
plt.show()

In [ ]:
fig, ax1 = plt.subplots()

t = list(range(len(true_states[trajectory_index])))

color = 'tab:red'
ax1.set_xlabel('Time step')
ax1.set_ylabel('True observations', color=color)
ax1.plot(t, X_observations[trajectory_index], color=color, linewidth=5, alpha=0.2)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  

color = 'tab:blue'
ax2.set_ylabel('Predicted average observations', color=color)  
ax2.plot(t, expected_observations[trajectory_index], color=color, linewidth=5, alpha=0.1)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yticks([0, 1, 2])

fig.tight_layout()  
plt.show()

In [ ]:
# Visualize the attention weights over time
import seaborn as sns

Attention_weights = []

for w in range(len(attention[0])):
    
    Attention_weights.append(np.vstack((attention[0][w], np.zeros((len(attention[0][-1]) - len(attention[0][w]),1)))))


Attention_weights = np.array(Attention_weights).reshape((len(attention[0][-1]), len(attention[0][-1])))[:state_inference[0].shape[0], :state_inference[0].shape[0]]

mask = np.zeros_like(Attention_weights)

mask[np.triu_indices_from(mask)] = True

with sns.axes_style("white"):
    ax = sns.heatmap(Attention_weights, mask=mask, vmax=.3, square=True)
    ax.set_ylabel('Chronological time')
    ax.set_xlabel('Previous time steps')

# TimeGANs Synthesized Data with ASSM Model

In [ ]:
# dataX[0]

In [ ]:
# possible order: Based on order in feat_list (generated when creating data) 46 items
# ['hs_health_conditions_skin' (0/1), 'hs_health_conditions_eye' (0/1), 'hs_health_conditions_cancer' (0/1), 
# 'dd_breed_pure_or_mixed' (1/2), 'df_ever_underweight' (1/0), 'hs_recent_hospitalization', 
# 'dd_spayed_or_neutered', 'dd_spay_or_neuter_age', 'dd_insurance', 
# 'de_daytime_sleep_avg_hours', 'de_nighttime_sleep_avg_hours', 'de_eats_feces', 
# 'df_appetite', 'df_appetite_change_last_year', 'df_ever_malnourished', 
# 'df_ever_overweight', 'df_feedings_per_day', 'free_fed', 'mp_flea_and_tick_treatment', 
# 'mp_heartworm_preventative', 'df_primary_diet_component', 'mp_dental_brushing_frequency', 
# 'db_excitement_level_before_walk', 'db_excitement_level_before_car_ride', 
# 'db_aggression_level_approached_while_eating', 'db_aggression_level_on_leash_unknown_dog', 
# 'db_left_alone_barking_frequency', 'db_left_alone_restlessness_frequency', 
# 'db_fear_level_loud_noises', 'db_fear_level_unknown_human_touch', 
# 'db_attention_seeking_follows_humans_frequency', 
# 'db_attention_seeking_sits_close_to_humans_frequency', 'hs_general_health', 
# 'dd_age_years', 'dd_sex', 'dd_weight_range', 'de_eats_grass_frequency', 
# 'db_training_distraction_frequency', 'db_training_obeys_stay_command_frequency', 
# 'db_barks_frequency' (0.0-4.0), 'db_escapes_home_or_property_frequency (0.0-4.0)', 'db_playful_frequency' (0.0-4.0), 
# 'db_urinates_in_home_frequency' (0.0-4.0), 'hs_follow_up_ongoing' (1/0), 
# 'hs_required_surgery_or_hospitalization' (1/2/3/4), 'dd_combined_main_breed' (0-300 or so)]

# another possibility: Based on order of df created from data creation: 47 items (including id)
# Index(['dog_id', 'db_aggression_level_approached_while_eating' (0.0 - 4.0),
#        'db_aggression_level_on_leash_unknown_dog (0.0 - 4.0)',
#        'db_attention_seeking_follows_humans_frequency',
#        'db_attention_seeking_sits_close_to_humans_frequency',
#        'db_barks_frequency', 'db_escapes_home_or_property_frequency',
#        'db_excitement_level_before_car_ride',
#        'db_excitement_level_before_walk', 'db_fear_level_loud_noises',
#        'db_fear_level_unknown_human_touch', 'db_left_alone_barking_frequency',
#        'db_left_alone_restlessness_frequency', 'db_playful_frequency',
#        'db_training_distraction_frequency',
#        'db_training_obeys_stay_command_frequency',
#        'db_urinates_in_home_frequency', 'dd_age_years',
#        'dd_breed_pure_or_mixed', 'dd_insurance', 'dd_sex',
#        'dd_spay_or_neuter_age', 'dd_spayed_or_neutered', 'dd_weight_range',
#        'de_daytime_sleep_avg_hours', 'de_eats_feces',
#        'de_eats_grass_frequency', 'de_nighttime_sleep_avg_hours',
#        'df_appetite', 'df_appetite_change_last_year', 'df_ever_malnourished',
#        'df_ever_overweight', 'df_ever_underweight', 'df_feedings_per_day',
#        'df_primary_diet_component', 'hs_general_health',
#        'hs_health_conditions_cancer', 'hs_health_conditions_eye',
#        'hs_health_conditions_skin', 'hs_recent_hospitalization',
#        'mp_dental_brushing_frequency', 'mp_flea_and_tick_treatment',
#        'mp_heartworm_preventative', 'hs_follow_up_ongoing',
#        'hs_required_surgery_or_hospitalization', 'dd_combined_main_breed',
#        'free_fed'],
#       dtype='object')

# first row output of synthesized data: (normalized!)
# 0.132518 (breed?) 0.0 (1-4?) 0.00 (1-4) 0.25 (1-4) 0.75 (1-4?) 0.0 (1-4?) 0.0 (1-4) 0.50 (1-4) 0.25 (1-4) 0.00 (1-4) ... 1.0 0.0 0.75 1.0 0.0 1.0 1.000000 0.010020 0.0

In [ ]:
# syn_df.sample(20)

In [ ]:
# syn_df = pd.DataFrame(dataX, columns=assm_feat_list) # ,dtype=float
list_of_lists = []

for arr in dataX_hat: #dataX
    # print(arr)
    for outer_lst in arr:
        # print(outer_lst)
        # print(outer_lst.shape)
        list_of_lists.append(outer_lst)
        # for d in outer_lst:
        #     print(d)
        
# print(len(assm_feat_list))
# print(assm_feat_list)
syn_df = pd.DataFrame(list_of_lists) # , columns=assm_feat_list)

# print(syn_df.shape)

class NumAdv:
    def __iter__(self):
        self.a = int(1)
        return self
    
    def __next__(self):
        x = self.a
        self.a += 1
        if self.a % 2==0:
            fin = (x / 2)
        else:
            fin = ((x - 1) / 2)
        return fin * 2

def get_id(df):
    dog_id = ((next(myiter)) + 100000) # original dog ids max at 92582
    return dog_id

# add a non-consecutive id for every 2 rows
idclass = NumAdv()
myiter = iter(idclass)
syn_df['dog_id'] = syn_df.apply(get_id, axis = 1)

syn_df.head(5)

In [ ]:
def generate_doggo_trajectory_syn(df, feats, predict='skin'):
    # print(assm_df.head(5))
    
    list_arr = []
    arry_list = np.array([])
    list_list = []
    list_feat = []
    cur_dog_id = df.iloc[0]['dog_id']
    
    df['state'] = 0.0
    state_list = []
    list_state_list = []
    
    for index, row in df.iterrows(): 
        # set up state lists - 'hs_health_conditions_?????'
        # start with just skin - 'hs_health_conditions_skin', 1 YES, 2 NO
        # set up state lists
        if row[39] <= 0.5: # 'hs_health_conditions_skin'
            row['state'] = 1
        elif row[39]  > 0.5: # 'hs_health_conditions_skin'
            row['state'] = 0
        
        if cur_dog_id != row['dog_id']:
            # print('list_list: ', list_list)
            arry_list = np.array(list_list)
            # print('arry_list: ', arry_list)
            list_arr.append(arry_list)
            list_state_list.append(state_list)
        
            # reset various fields
            state_list = []
            arry_list = np.array([]) # shouldn't need this, but jic
            list_list = [] # reset the list of lists
            cur_dog_id = row['dog_id']

        # set up features list:
        # hard code this for now, then go back and iterate it:
        # [‘hs_general_health','hs_recent_hospitalization']
        # [‘hs_follow_up_ongoing',‘hs_required_surgery_or_hospitalization’]
        # ['hs_health_conditions_skin','hs_health_conditions_eye','hs_health_conditions_cancer']
        
#         list_feat = [row['dis_skin'],row['dis_oral'], row['dis_trau'], row['dis_infe'], row['dis_bone'], row['dis_eyes']]
        # list_feat = [row['hs_general_health'],row['hs_recent_hospitalization'], row['hs_follow_up_ongoing'], row['hs_required_surgery_or_hospitalization'], row['hs_health_conditions_eye'], row['hs_health_conditions_cancer']]
        # print('pre feats list_feat', list_feat)
        # print(feats)
        for feat in range(0,39):
            # print(feat)
            list_feat.append(row[feat])
        # have to skip skin
        for feat in range(40,46):
            # print(feat)
            list_feat.append(row[feat])
            
        # print('list_feat: ', list_feat)
        # print('list_feat size: ', len(list_feat))
        list_list.append(list_feat)
        state_list.append(row[39])
        list_feat = [] # shouldn't need this, but jic
    
    # if predict=='skin':
    #     drop_cols = ['hs_health_conditions_skin']
    #     df.drop(drop_cols, axis=1)
    # elif predict=='eye':
    #     drop_cols = ['hs_health_conditions_eye']
    #     df.drop(drop_cols, axis=1)            
    # elif predict=='cancer':
    #     drop_cols = ['hs_health_conditions_cancer']
    #     df.drop(drop_cols, axis=1)
    
    # load final vars
    X_ = list_arr
    # arrX = np.array(X_)
    # print('x shape: ', arrX.shape)
    
    S_ = list_state_list
    # arrS = np.array(S_)
    # print('x shape: ', arrS.shape)
    
#     print(arrX[0:20])
#     print(arrS[0:20])
    
    return X_, S_ 

In [ ]:
## Data Loading

# FYI - can't get entire DF to run through without ASSM collapsing.  Max on minimum params is 21,000, larger params 29,400
X_observations_syn, true_states_syn = generate_doggo_trajectory_syn(syn_df[0:17300], feats=assm_feat_list, predict=disease)

# TODO adjust 0:20000 to as high as possible
# keep adding categories!

# assm_df.head(10)
# X_observations[0:3]
# true_states[0:10]

# ## Data Loading
# seq_length = 4
# data_cats = [1]

# df, feat_num, feat_list = doggo_data_loading(seq_length=seq_length, data_cat=data_cats)

In [ ]:
trajectory_index_syn = 3

true_states_syn[trajectory_index_syn]

In [ ]:
# Visualizing the hidden state data trajectories
fig, ax1 = plt.subplots()

t_syn = list(range(len(true_states_syn[trajectory_index_syn])))

color = 'tab:red'
ax1.set_xlabel('Time step')
ax1.set_ylabel('Observations', color=color)
ax1.plot(t_syn, X_observations_syn[trajectory_index_syn], color=color, linewidth=5, alpha=0.2)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  

color = 'tab:blue'
ax2.set_ylabel('Hidden states', color=color)  
ax2.step(t_syn, true_states_syn[trajectory_index_syn], color=color, linewidth=5)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yticks([0, 1, 2])

fig.tight_layout()  
plt.show()

In [ ]:
print(assm_feat_num)
model_syn = attentive_state_space_model(num_states=2,
                              maximum_seq_length=30, 
                              input_dim=assm_feat_num-1, # 6,  # started with: 6. tried 10, but broke, but that was original, adjust for how many features used in data
                              rnn_type='LSTM',
                              latent=True,
                              generative=True,
                              num_iterations=50,   # started with: 5
                              num_epochs=3,  # started with: 2
                              batch_size=100,  # started with: 10
                              learning_rate=5*1e-4, 
                              num_rnn_hidden=100,  # started with: 5
                              num_rnn_layers=1,
                              dropout_keep_prob=None,
                              num_out_hidden=100,  # started with: 5
                              num_out_layers=1)


In [ ]:
model_syn.fit(X_observations_syn)

In [ ]:
model_syn.initial_probabilities

In [ ]:
model_syn.transition_matrix

In [ ]:
model_syn.state_means.T 

In [ ]:
model_syn.state_covars

In [ ]:
X_observations_syn[3]

In [ ]:
predict_idx_syn = 0
state_inference_syn, expected_observations_syn, attention_syn = model_syn.predict([X_observations_syn[predict_idx_syn]])

In [ ]:
np.argmax(state_inference_syn[predict_idx_syn], axis=1)

In [ ]:
fig, ax1 = plt.subplots()
trajectory_index_syn = 0
t_syn = list(range(len(true_states_syn[trajectory_index_syn])))
color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('MAP state estimate', color=color)
ax1.plot(t_syn, np.argmax(state_inference_syn[trajectory_index_syn], axis=1), color=color, linewidth=5)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_yticks([0, 1, 2])
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('True states', color=color)  
ax2.step(t_syn, true_states_syn[trajectory_index_syn], color=color, linewidth=5)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yticks([0, 1, 2])
fig.tight_layout()  
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
t_syn = list(range(len(true_states_syn[trajectory_index_syn])))
color = 'tab:red'
ax1.set_xlabel('Time step')
ax1.set_ylabel('True observations', color=color)
ax1.plot(t_syn, X_observations_syn[trajectory_index_syn], color=color, linewidth=5, alpha=0.2)
ax1.tick_params(axis='y', labelcolor=color)
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('Predicted average observations', color=color)  
ax2.plot(t_syn, expected_observations_syn[trajectory_index_syn], color=color, linewidth=5, alpha=0.1)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yticks([0, 1, 2])
fig.tight_layout()  
plt.show()

In [ ]:
# Visualize the attention weights over time
import seaborn as sns
Attention_weights_syn = []
for w in range(len(attention_syn[0])):
    Attention_weights_syn.append(np.vstack((attention_syn[0][w], np.zeros((len(attention_syn[0][-1]) - len(attention_syn[0][w]),1)))))
Attention_weights_syn = np.array(Attention_weights_syn).reshape((len(attention_syn[0][-1]), len(attention_syn[0][-1])))[:state_inference_syn[0].shape[0], :state_inference_syn[0].shape[0]]
mask_syn = np.zeros_like(Attention_weights_syn)
mask_syn[np.triu_indices_from(mask_syn)] = True
with sns.axes_style("white"):
    ax = sns.heatmap(Attention_weights_syn, mask=mask_syn, vmax=.3, square=True)
    ax.set_ylabel('Chronological time')
    ax.set_xlabel('Previous time steps')

# Using TimeGANs to Oversample Minority Data Class

In [ ]:
# Experiments iterations
Iteration = 2
Sub_Iteration = 3

# take original TimeGANs loaded data as usual, 
#   but then filter by 'hs_health_conditions_skin'==1
#   before passing into TimeGANs 
print('current df size: ', gan_df.shape)
gan_df_min = gan_df[gan_df['hs_health_conditions_skin'] == 1]
print('new df size: ', gan_df_min.shape) # should be about 85% smaller?

# so run original timeGANS data changer, but on smaller df
dataX_min = doggo_data_timeGANs(gan_df_min)

print('minority doggo dataset is ready')
# df.sample(28)
# df.head(28)
gan_df_min.head(10)

In [ ]:
## Newtork Parameters
parameters_min = dict()

parameters_min['hidden_dim'] = len(dataX_min[0][0,:]) * 4
parameters_min['num_layers'] = 3 # started with: 2 (has to be at least 2!)
parameters_min['iterations'] = 10000 # started with: 1000, low as 10
parameters_min['batch_size'] = 128 # started with: 64, low as 2
parameters_min['module_name'] = 'gru'   # Other options: 'lstm' or 'lstmLN'
parameters_min['z_dim'] = len(dataX_min[0][0,:]) 

print('Minority class parameters are ' + str(parameters_min))

## Experiments
# Output Initialization
Discriminative_Score_min = list()
Predictive_Score_min = list()

In [ ]:
# Run TimeGANs model
print('Start iterations') 
    
# Each Iteration
for it in range(Iteration):

    
    # Synthetic Data Generation
    dataX_hat_min = timegan(dataX_min, parameters_min)   
      
    print('Finish Synthetic Data Generation')

    ## Performance Metrics
    
    # 1. Discriminative Score
    Acc_min = list()
    for tt in range(Sub_Iteration):
        Temp_Disc_min = discriminative_score_metrics(dataX_min, dataX_hat_min)
        Acc_min.append(Temp_Disc_min)
    
    Discriminative_Score_min.append(np.mean(Acc_min))
    
    # 2. Predictive Performance
    MAE_All_min = list()
    for tt in range(Sub_Iteration):
        MAE_All_min.append(predictive_score_metrics(dataX_min, dataX_hat_min))
        
    Predictive_Score_min.append(np.mean(MAE_All_min))    
    
print('Finish TimeGAN iterations')

In [ ]:
PCA_Analysis(dataX_min, dataX_hat_min)

In [ ]:
tSNE_Analysis(dataX_min, dataX_hat_min)

In [ ]:
# Print Results
print('Minority class discriminative Score - Mean: ' + str(np.round(np.mean(Discriminative_Score_min),4)) + ', Std: ' + str(np.round(np.std(Discriminative_Score_min),4)))
print('Minority class  predictive Score - Mean: ' + str(np.round(np.mean(Predictive_Score_min),4)) + ', Std: ' + str(np.round(np.std(Predictive_Score_min),4)))

In [ ]:
print('new dogs: ', len(dataX_hat_min))
print('meet scruffy: ', dataX_hat_min[0])

In [ ]:
seq_length = 4
data_cats = [1,2,3,4,5,6,7,8,9,10,11,12,13] # 
disease = 'skin'

# generate the data
# df, feat_num, feat_list = doggo_data_loading(seq_length=seq_length, data_cat=data_cats) # [1,2,3,4,5,6,7,8,9,10,11,12,13] # [1,10,14]
# final_df, feat_num, feat_list = doggo_data_loading(apply_seq_length=False, use_surg=False, data_cat=data_cats)
orig_df, orig_feat_num, orig_feat_list = doggo_data_loading(seq_length=seq_length, apply_seq_length=True, data_cat=data_cats) #  apply_seq_length=True, use_surg=False, 
print(orig_feat_num, ' : ', orig_feat_list)

print(orig_df.shape)
orig_df.head(10)

In [ ]:
# col = "dog_id"
# max_x = orig_df.loc[orig_df[col].idxmax()]
# print("Maximum value of column ", col, " and its corresponding row values:\n", max_x)

In [ ]:
# # sync_data is the dataX_hat output from the TimeGANs
# list_of_lists = []
# for arr in dataX_hat_min:  # use sync_data in func
# # print(arr)
#     for outer_lst in arr:
#         # print(outer_lst)
#         # print(outer_lst.shape)
#         list_of_lists.append(outer_lst)

# sync_df = pd.DataFrame(list_of_lists)

# idclass = NumAdv()
# myiter = iter(idclass)
# sync_df['dog_id'] = sync_df.apply(get_id, axis = 1)

# # get a list of columns
# cols = list(sync_df)
# cols.insert(0, cols.pop(cols.index('dog_id')))
# # cols

# sync_df = sync_df.loc[:, cols]
# # # sync_df

# print(orig_df.columns)
# print(len(orig_df.columns))
# sync_df.columns = orig_df.columns

# print(sync_df.shape)
# sync_df.head(10)

# MOVED TO FUNC


In [ ]:
# combine new datapoints with existing dataset
def generate_doggo_trajectory_comb(orig_df, sync_data, feats, predict='skin'):
    # print(assm_df.head(5))
    
    # syn_data is the dataX_hat output from the TimeGANs
    list_of_lists = []
    for arr in sync_data:
        # print(arr)
        for outer_lst in arr:
            # print(outer_lst)
            # print(outer_lst.shape)
            list_of_lists.append(outer_lst)
        
    sync_df = pd.DataFrame(list_of_lists)

    idclass = NumAdv()
    myiter = iter(idclass)
    sync_df['dog_id'] = sync_df.apply(get_id, axis = 1)

    # move dog_id to front of columns to align with orig_df
    cols = list(sync_df)
    cols.insert(0, cols.pop(cols.index('dog_id')))
    sync_df = sync_df.loc[:, cols]

    # assign col names to sync data
    sync_df.columns = orig_df.columns

    # join the 2 dataframes
    comb_df = pd.concat([orig_df, sync_df])
    print('combined df size: ', comb_df.shape)
    
    list_arr = []
    arry_list = np.array([])
    list_list = []
    list_feat = []
    cur_dog_id = comb_df.iloc[0]['dog_id']
    
    comb_df['state'] = 0.0
    state_list = []
    list_state_list = []
    
    for index, row in comb_df.iterrows(): 
        # set up state lists - 'hs_health_conditions_?????'
        # start with just skin - 'hs_health_conditions_skin', 1 YES, 2 NO
        # set up state lists
        if row['hs_health_conditions_skin'] <= 0.5: # 'hs_health_conditions_skin'
            row['state'] = 1
        elif row['hs_health_conditions_skin']  > 0.5: # 'hs_health_conditions_skin'
            row['state'] = 0
        
        if cur_dog_id != row['dog_id']:
            # print('list_list: ', list_list)
            arry_list = np.array(list_list)
            # print('arry_list: ', arry_list)
            list_arr.append(arry_list)
            list_state_list.append(state_list)
        
            # reset various fields
            state_list = []
            arry_list = np.array([]) # shouldn't need this, but jic
            list_list = [] # reset the list of lists
            cur_dog_id = row['dog_id']

        # set up features list:
        # hard code this for now, then go back and iterate it:
        # [‘hs_general_health','hs_recent_hospitalization']
        # [‘hs_follow_up_ongoing',‘hs_required_surgery_or_hospitalization’]
        # ['hs_health_conditions_skin','hs_health_conditions_eye','hs_health_conditions_cancer']
        
#         list_feat = [row['dis_skin'],row['dis_oral'], row['dis_trau'], row['dis_infe'], row['dis_bone'], row['dis_eyes']]
        # list_feat = [row['hs_general_health'],row['hs_recent_hospitalization'], row['hs_follow_up_ongoing'], row['hs_required_surgery_or_hospitalization'], row['hs_health_conditions_eye'], row['hs_health_conditions_cancer']]
        # print('pre feats list_feat', list_feat)
        # print(feats)
        for feat in feats:
            # print(feat)
            list_feat.append(row[feat])
        # print('list_feat: ', list_feat)
        # print('list_feat size: ', len(list_feat))
        list_list.append(list_feat)
        state_list.append(row['state'])
        list_feat = [] # shouldn't need this, but jic
    
    # if predict=='skin':
    #     drop_cols = ['hs_health_conditions_skin']
    #     df.drop(drop_cols, axis=1)
    # elif predict=='eye':
    #     drop_cols = ['hs_health_conditions_eye']
    #     df.drop(drop_cols, axis=1)            
    # elif predict=='cancer':
    #     drop_cols = ['hs_health_conditions_cancer']
    #     df.drop(drop_cols, axis=1)
    
    # load final vars
    X_ = list_arr
    # arrX = np.array(X_)
    # print('x shape: ', arrX.shape)
    
    S_ = list_state_list
    # arrS = np.array(S_)
    # print('x shape: ', arrS.shape)
    
#     print(arrX[0:20])
#     print(arrS[0:20])
    
    return X_, S_ 

In [ ]:
# FYI - can't get entire DF to run through without ASSM collapsing.  Max on minimum params is 21,000, larger params 29,400
# undersample the majority case. keep all minority case:
# TODO come back to this!

# combine and form data for model
X_observations_comb, true_states_comb = generate_doggo_trajectory_comb(orig_df[0:17000], dataX_hat_min, feats=assm_feat_list, predict=disease)

# TODO adjust 0:20000 to as high as possible
# keep adding categories!

# assm_df.head(20)
# X_observations[0:3]
# true_states[0:10]

# ## Data Loading
# seq_length = 4
# data_cats = [1]

# df, feat_num, feat_list = doggo_data_loading(seq_length=seq_length, data_cat=data_cats)

In [ ]:
trajectory_index_comb = 3

true_states_comb[trajectory_index_comb]

In [ ]:
# Visualizing the hidden state data trajectories
fig, ax1 = plt.subplots()

t_comb = list(range(len(true_states_comb[trajectory_index_comb])))

color = 'tab:red'
ax1.set_xlabel('Time step')
ax1.set_ylabel('Observations', color=color)
ax1.plot(t_comb, X_observations_comb[trajectory_index_comb], color=color, linewidth=5, alpha=0.2)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  

color = 'tab:blue'
ax2.set_ylabel('Hidden states', color=color)  
ax2.step(t_comb, true_states_comb[trajectory_index_comb], color=color, linewidth=5)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yticks([0, 1, 2])

fig.tight_layout()  
plt.show()

In [ ]:
dims = orig_feat_num
print(dims)
model_comb = attentive_state_space_model(num_states=2,
                              maximum_seq_length=30, 
                              input_dim=dims, # 6,  # started with: 6. tried 10, but broke, but that was original, adjust for how many features used in data
                              rnn_type='LSTM',
                              latent=True,
                              generative=True,
                              num_iterations=50,   # started with: 5
                              num_epochs=3,  # started with: 2
                              batch_size=100,  # started with: 10
                              learning_rate=5*1e-4, 
                              num_rnn_hidden=100,  # started with: 5
                              num_rnn_layers=1,
                              dropout_keep_prob=None,
                              num_out_hidden=100,  # started with: 5
                              num_out_layers=1)


In [ ]:
model_comb.fit(X_observations_comb)

In [ ]:
model_comb.initial_probabilities

In [ ]:
model_comb.transition_matrix

In [ ]:
model_comb.state_means.T 

In [ ]:
model_comb.state_covars

In [ ]:
X_observations_comb[3]

In [ ]:
predict_idx_comb = 0
state_inference_comb, expected_observations_comb, attention_comb = model_comb.predict([X_observations_comb[predict_idx_comb]])

In [ ]:
np.argmax(state_inference_comb[predict_idx_comb], axis=1)

In [ ]:
fig, ax1 = plt.subplots()
trajectory_index_comb = 0
t_comb = list(range(len(true_states_comb[trajectory_index_comb])))
color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('MAP state estimate', color=color)
ax1.plot(t_comb, np.argmax(state_inference_comb[trajectory_index_comb], axis=1), color=color, linewidth=5)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_yticks([0, 1, 2])
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('True states', color=color)  
ax2.step(t_comb, true_states_comb[trajectory_index_comb], color=color, linewidth=5)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yticks([0, 1, 2])
fig.tight_layout()  
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
t_comb = list(range(len(true_states_comb[trajectory_index_comb])))
color = 'tab:red'
ax1.set_xlabel('Time step')
ax1.set_ylabel('True observations', color=color)
ax1.plot(t_comb, X_observations_comb[trajectory_index_comb], color=color, linewidth=5, alpha=0.2)
ax1.tick_params(axis='y', labelcolor=color)
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('Predicted average observations', color=color)  
ax2.plot(t_comb, expected_observations_comb[trajectory_index_comb], color=color, linewidth=5, alpha=0.1)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yticks([0, 1, 2])
fig.tight_layout()  
plt.show()

In [ ]:
# Visualize the attention weights over time
import seaborn as sns
Attention_weights_comb = []
for w in range(len(attention_comb[0])):
    Attention_weights_comb.append(np.vstack((attention_comb[0][w], np.zeros((len(attention_comb[0][-1]) - len(attention_comb[0][w]),1)))))
Attention_weights_comb = np.array(Attention_weights_comb).reshape((len(attention_comb[0][-1]), len(attention_comb[0][-1])))[:state_inference_comb[0].shape[0], :state_inference_comb[0].shape[0]]
mask_comb = np.zeros_like(Attention_weights_comb)
mask_comb[np.triu_indices_from(mask_comb)] = True
with sns.axes_style("white"):
    ax = sns.heatmap(Attention_weights_comb, mask=mask_comb, vmax=.3, square=True)
    ax.set_ylabel('Chronological time')
    ax.set_xlabel('Previous time steps')